# Marketing Budget Optimisation

## Market mix model to observe the actual impact of different marketing variables

## Problem Statement
ElecKart is an e-commerce firm based out of Ontario, Canada specialising in electronic products. Over the last one year, they had spent a significant amount of money on marketing. Occasionally, they had also offered big-ticket promotions (similar to the Big Billion Day). They are about to create a marketing budget for the next year, which includes spending on commercials, online campaigns, and pricing & promotion strategies. The CFO feels that the money spent over the last 12 months on marketing was not sufficiently impactful, and, that they can either cut on the budget or reallocate it optimally across marketing levers to improve the revenue response.

Imagine that you are a part of the marketing team working on budget optimisation. You need to develop a market mix model to observe the actual impact of different marketing variables over the last year. Using your understanding of the model, you have to recommend the optimal budget allocation for different marketing levers for the next year.

## 1. Data Reading And Understanding

In [1]:
import warnings
warnings.filterwarnings('ignore')

#importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_profiling
import seaborn as sns
from datetime import datetime
pd.set_option('display.float_format', '{:0.3f}'.format)

# !pip install termcolor
import colorama
from colorama import Fore, Style  # maakes strings colored
from termcolor import colored
from termcolor import cprint

In [2]:
# Read the file
consumer = pd.read_csv('ConsumerElectronics.csv')

In [3]:
consumer.head()

,fsn_id,order_date,Year,Month,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,s1_fact.order_payment_type,sla,cust_id,pincode,product_analytic_super_category,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp,product_procurement_sla
0,ACCCX3S58G7B5F6P,2015-10-17 15:11:54,2015,10,3419300926147000.000,3419300926147000.000,6400,1,\N,\N,COD,5,-1.01299130778588E+018,-7.79175582905735E+018,CE,CameraAccessory,CameraAccessory,CameraTripod,7190,0
1,ACCCX3S58G7B5F6P,2015-10-19 10:07:22,2015,10,1420830839915200.000,1420830839915200.000,6900,1,\N,\N,COD,7,-8.99032457905512E+018,7.33541149097431E+018,CE,CameraAccessory,CameraAccessory,CameraTripod,7190,0
2,ACCCX3S5AHMF55FV,2015-10-20 15:45:56,2015,10,2421912925714800.000,2421912925714800.000,1990,1,\N,\N,COD,10,-1.0404429420466E+018,-7.47768776228657E+018,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3
3,ACCCX3S5AHMF55FV,2015-10-14 12:05:15,2015,10,4416592101738400.000,4416592101738400.000,1690,1,\N,\N,Prepaid,4,-7.60496084352714E+018,-5.83593163877661E+018,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3
4,ACCCX3S5AHMF55FV,2015-10-17 21:25:03,2015,10,4419525153426400.000,4419525153426400.000,1618,1,\N,\N,Prepaid,6,2.8945572083453E+018,5.34735360997242E+017,CE,CameraAccessory,CameraAccessory,CameraTripod,2099,3


In [4]:
consumer.shape

(1648824, 20)

In [5]:
# Get info about the dataset
consumer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1648824 entries, 0 to 1648823
Data columns (total 20 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   fsn_id                           1648824 non-null  object 
 1   order_date                       1648824 non-null  object 
 2   Year                             1648824 non-null  int64  
 3   Month                            1648824 non-null  int64  
 4   order_id                         1648824 non-null  float64
 5   order_item_id                    1648824 non-null  float64
 6   gmv                              1648824 non-null  object 
 7   units                            1648824 non-null  int64  
 8   deliverybdays                    1648824 non-null  object 
 9   deliverycdays                    1648824 non-null  object 
 10  s1_fact.order_payment_type       1648824 non-null  object 
 11  sla                              1648824 non-null 

In [6]:
# consumer.profile_report()

In [7]:
def missing_values(consumer):
    missing_number = consumer.isnull().sum().sort_values(ascending = False)
    missing_percent = (consumer.isnull().sum() / consumer.isnull().count()).sort_values(ascending = False)
    missing_values = pd.concat([missing_number, missing_percent], axis = 1, keys = ['Missing_Number', 'Missing_Percent'])
    return missing_values[missing_values['Missing_Number'] > 0]

def first_looking(consumer):
    print(colored("Shape:", attrs=['bold']), consumer.shape,'\n', 
          colored('*'*100, 'red', attrs = ['bold']),
          colored("\nInfo:\n", attrs = ['bold']), sep = '')
    print(consumer.info(), '\n', 
          colored('*'*100, 'red', attrs = ['bold']), sep = '')
    print(colored("Number of Uniques:\n", attrs = ['bold']), consumer.nunique(),'\n',
          colored('*'*100, 'red', attrs = ['bold']), sep = '')
    print(colored("Missing Values:\n", attrs=['bold']), missing_values(consumer),'\n', 
          colored('*'*100, 'red', attrs = ['bold']), sep = '')
    print(colored("All Columns:", attrs = ['bold']), list(consumer.columns),'\n', 
          colored('*'*100, 'red', attrs = ['bold']), sep = '')

    consumer.columns = consumer.columns.str.lower().str.replace('&', '_').str.replace(' ', '_')
    print(colored("Columns after rename:", attrs = ['bold']), list(consumer.columns),'\n',
          colored('*'*100, 'red', attrs = ['bold']), sep = '')  
    print(colored("Columns after rename:", attrs = ['bold']), list(consumer.columns),'\n',
          colored('*'*100, 'red', attrs = ['bold']), sep = '')
    print(colored("Descriptive Statistics \n", attrs = ['bold']), consumer.describe().round(2),'\n',
          colored('*'*100, 'red', attrs = ['bold']), sep = '') # Gives a statstical breakdown of the data.
    print(colored("Descriptive Statistics (Categorical Columns) \n", attrs = ['bold']), consumer.describe(include = object).T,'\n',
          colored('*'*100, 'red', attrs = ['bold']), sep = '') # Gives a statstical breakdown of the data.

def duplicate_values(consumer):
    print(colored("Duplicate check...", attrs = ['bold']), sep = '')
    print("There are", consumer.duplicated(subset = None, keep = 'first').sum(), "duplicated observations in the dataset.")
    duplicate_values = consumer.duplicated(subset = None, keep = 'first').sum()
    if duplicate_values > 0:
        consumer.drop_duplicates(keep = 'first', inplace = True)
        print(duplicate_values, colored(" Duplicates were dropped!"),'\n',
              colored('*'*100, 'red', attrs = ['bold']), sep = '')

In [8]:
# first_looking(consumer)
# duplicate_values(consumer)
# print(colored("Shape:", attrs = ['bold']), consumer.shape,'\n', colored('*'*100, 'red', attrs = ['bold']))

In [9]:
consumer.describe()

,Year,Month,order_id,order_item_id,units,sla,product_mrp,product_procurement_sla
count,1648824.000,1648824.000,1648824.000,1648824.000,1648824.000,1648824.000,1648824.000,1648824.000
mean,2015.519,6.586,2842444674146328.500,13756277530325384.000,1.022,5.688,4108.309,5.403
std,0.500,3.623,1269134332596894.250,55945981798631008.000,0.254,2.983,8510.918,53.466
min,2015.000,1.000,76331515.000,108016495.000,1.000,0.000,0.000,-1.000
25%,2015.000,3.000,1602588503969575.000,1606725138262300.000,1.000,4.000,799.000,1.000
50%,2016.000,6.000,2611824528743350.000,2619185472741150.000,1.000,6.000,1599.000,2.000
75%,2016.000,10.000,3610613539366250.000,4397705931628400.000,1.000,7.000,3499.000,3.000
max,2016.000,12.000,5395076790260600.000,300037347717704000.000,50.000,1006.000,299999.000,1000.000


In [10]:
consumer.isnull().sum()

fsn_id                             0
order_date                         0
Year                               0
Month                              0
order_id                           0
order_item_id                      0
gmv                                0
units                              0
deliverybdays                      0
deliverycdays                      0
s1_fact.order_payment_type         0
sla                                0
cust_id                            0
pincode                            0
product_analytic_super_category    0
product_analytic_category          0
product_analytic_sub_category      0
product_analytic_vertical          0
product_mrp                        0
product_procurement_sla            0
dtype: int64

- There are no null values in the dataset!
- Columns like order_date, gmv, deliverybdays, deliverycdays, pincode have incorrect data types and need to be changed.

In [11]:
# Let's take a look at the statistical info of the dataset
consumer.describe(percentiles = [0.25, 0.5, 0.75, 0.90, 0.99, 0.999])

,Year,Month,order_id,order_item_id,units,sla,product_mrp,product_procurement_sla
count,1648824.000,1648824.000,1648824.000,1648824.000,1648824.000,1648824.000,1648824.000,1648824.000
mean,2015.519,6.586,2842444674146328.500,13756277530325384.000,1.022,5.688,4108.309,5.403
std,0.500,3.623,1269134332596894.250,55945981798631008.000,0.254,2.983,8510.918,53.466
min,2015.000,1.000,76331515.000,108016495.000,1.000,0.000,0.000,-1.000
25%,2015.000,3.000,1602588503969575.000,1606725138262300.000,1.000,4.000,799.000,1.000
50%,2016.000,6.000,2611824528743350.000,2619185472741150.000,1.000,6.000,1599.000,2.000
75%,2016.000,10.000,3610613539366250.000,4397705931628400.000,1.000,7.000,3499.000,3.000
90%,2016.000,11.000,4520541993801910.000,4561094519921880.000,1.000,9.000,7120.000,5.000
99%,2016.000,12.000,4626530348261358.000,300037347714467776.000,2.000,13.000,45990.000,13.000
99.9%,2016.000,12.000,4639920775381027.000,300037347717600000.000,4.000,17.000,54950.000,1000.000


- All the numeric columns are almost normally distributed!
- We need to cap the SLAs to max and min values.
## 2. Data Cleaning And Preparation

In [12]:
consumer.columns

Index(['fsn_id', 'order_date', 'Year', 'Month', 'order_id', 'order_item_id',
       'gmv', 'units', 'deliverybdays', 'deliverycdays',
       's1_fact.order_payment_type', 'sla', 'cust_id', 'pincode',
       'product_analytic_super_category', 'product_analytic_category',
       'product_analytic_sub_category', 'product_analytic_vertical',
       'product_mrp', 'product_procurement_sla'],
      dtype='object')

In [13]:
consumer.replace(r'^\s+$', np.nan, regex=True, inplace = True)
consumer.replace('\\N', np.nan, inplace = True)

In [14]:
# let's check the null percentage for each column
round(100*(consumer.isnull().sum()/len(consumer.index)), 2)

fsn_id                             0.000
order_date                         0.000
Year                               0.000
Month                              0.000
order_id                           0.000
order_item_id                      0.000
gmv                                0.300
units                              0.000
deliverybdays                     79.630
deliverycdays                     79.630
s1_fact.order_payment_type         0.000
sla                                0.000
cust_id                            0.300
pincode                            0.300
product_analytic_super_category    0.000
product_analytic_category          0.000
product_analytic_sub_category      0.000
product_analytic_vertical          0.350
product_mrp                        0.000
product_procurement_sla            0.000
dtype: float64

In [15]:
#removing null valued GMV
consumer = consumer.loc[~(consumer.gmv.isnull())]

In [16]:
# let's check the null percentage for each column again
round(100*(consumer.isnull().sum()/len(consumer.index)), 2)

fsn_id                             0.000
order_date                         0.000
Year                               0.000
Month                              0.000
order_id                           0.000
order_item_id                      0.000
gmv                                0.000
units                              0.000
deliverybdays                     79.570
deliverycdays                     79.570
s1_fact.order_payment_type         0.000
sla                                0.000
cust_id                            0.000
pincode                            0.000
product_analytic_super_category    0.000
product_analytic_category          0.000
product_analytic_sub_category      0.000
product_analytic_vertical          0.350
product_mrp                        0.000
product_procurement_sla            0.000
dtype: float64

### 1. product_analytic_super_category, product_analytic_category, product_analytic_sub_category, product_analytic_vertical

In [17]:
# Let's drop the rows that have product analytic vertical as null.
consumer = consumer[~pd.isnull(consumer.product_analytic_vertical)]

In [18]:
# Let's now check the product_analytic_super_category unique values
consumer.product_analytic_super_category.unique()

array(['CE'], dtype=object)

There is only one value for this column. Hence, we can remove the column.

In [19]:
consumer.drop('product_analytic_super_category',1, inplace = True)

In [20]:
consumer.product_analytic_category.unique()

array(['CameraAccessory', 'GamingHardware', 'EntertainmentSmall',
       'GameCDDVD', 'Camera'], dtype=object)

In [21]:
consumer.product_analytic_sub_category.unique()

array(['CameraAccessory', 'GamingAccessory', 'Speaker', 'HomeAudio',
       'CameraStorage', 'AmplifierReceiver', 'AudioMP3Player', 'Game',
       'Camera', 'GamingConsole', 'HomeTheatre', 'AudioAccessory',
       'TVVideoSmall', 'GameMembershipCards'], dtype=object)

In [22]:
#The three product sub categories for the MMM are - camera accessory, home audio and gaming accessory.
#Removing the rows with other sub categories

consumer = consumer.loc[(consumer.product_analytic_sub_category=='CameraAccessory') |
                       (consumer.product_analytic_sub_category=='GamingAccessory')|
                       (consumer.product_analytic_sub_category=='HomeAudio')]

In [23]:
consumer.product_analytic_vertical.unique()

array(['CameraTripod', 'Lens', 'CameraBag', 'Flash', 'CameraBattery',
       'CameraBatteryCharger', 'GamePad', 'GamingAccessoryKit',
       'GamingHeadset', 'GamingMemoryCard', 'MotionController',
       'GamingMouse', 'GamingAdapter', 'TVOutCableAccessory', 'GamingGun',
       'JoystickGamingWheel', 'HomeAudioSpeaker', 'CameraRemoteControl',
       'Binoculars', 'Filter', 'GamingMousePad', 'Strap',
       'GamingKeyboard', 'Dock', 'SlingBox', 'GamingSpeaker',
       'CoolingPad', 'BoomBox', 'CameraBatteryGrip', 'CameraAccessory',
       'CameraFilmRolls', 'CameraMount', 'DockingStation', 'DJController',
       'ExtensionTube', 'FMRadio', 'HiFiSystem', 'KaraokePlayer',
       'Telescope', 'VoiceRecorder', 'GamingChargingStation',
       'CameraEyeCup', 'GameControlMount', 'SoundMixer', 'CameraLEDLight',
       'CameraMicrophone', 'FlashShoeAdapter', 'Softbox',
       'ReflectorUmbrella', 'Teleconverter', 'CameraHousing'],
      dtype=object)

### 2. gmv

In [24]:
#Let's convert the data type of GMV

consumer['gmv'] = pd.to_numeric(consumer['gmv'])

In [25]:
#Checking the minimum and maximum values of GMV
print(consumer.gmv.min())
print(consumer.gmv.max())

0.0
148050.0


gmv (Gross Merchendising Value - The cost price at which the item is sold multiplied by number of units) - Value at 0.0 seems odd.

Assumption : It could be because of any promotional offers, hence not deleting them

Looks like a duplicated order. Let's check for duplicates

In [26]:
consumer[consumer.duplicated(['fsn_id','order_date','order_id','order_item_id',
                              'gmv','cust_id','pincode',
                              'product_analytic_category','product_analytic_sub_category',
                             'product_analytic_vertical'])]
#consumer.loc[consumer.duplicated()]

,fsn_id,order_date,Year,Month,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,s1_fact.order_payment_type,sla,cust_id,pincode,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp,product_procurement_sla
76506,ACCE6YVWVHXXKCHW,2016-02-22 14:52:12,2016,2,3417290135521600.000,3417290135521600.000,350.000,1,7,8,Prepaid,8,-1.07035781735315E+018,8.45640885616433E+018,GamingHardware,GamingAccessory,JoystickGamingWheel,599,2
675305,ACCE7AV9TZHHAESC,2016-04-27 07:03:09,2016,4,2532857872757300.000,2532857872757300.000,4090.000,1,2,3,Prepaid,5,233114630667481984.000,-4071447699433639936.000,GamingHardware,GamingAccessory,GamePad,4990,2
944830,ACCDH7MFZJGRWJCW,2016-04-27 00:19:25,2016,4,3575842474884000.000,300037347717633984.000,287.000,1,5,6,Prepaid,5,8788006738130379776.000,2175163142166510080.000,GamingHardware,GamingAccessory,GamePad,410,1
944903,ACCDH7MFZJGRWJCW,2016-04-27 00:19:25,2016,4,3575842474884000.000,3575842474884000.000,287.000,1,NaN,NaN,Prepaid,7,8788006738130379776.000,2175163142166510080.000,GamingHardware,GamingAccessory,GamePad,410,1
969493,ACCE94WYZR6GFDFP,2016-04-27 00:33:53,2016,4,1569690464156100.000,1569690464156100.000,190.000,1,NaN,NaN,Prepaid,6,6615160307439460352.000,-7307608120567170048.000,CameraAccessory,CameraAccessory,Filter,380,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1647201,VOREYV2HWZDUFZ9Y,2015-09-29 23:19:16,2015,9,4404040347013500.000,4404040347013500.000,5200.000,1,NaN,NaN,Prepaid,11,-8.7451731063629E+018,3.1115855848644E+018,EntertainmentSmall,HomeAudio,VoiceRecorder,10500,5
1647202,VOREYV2HWZDUFZ9Y,2015-09-30 22:08:55,2015,9,2404860211262700.000,2404860211262700.000,5200.000,1,NaN,NaN,COD,10,-8.90379286900787E+018,-5.60164751149697E+018,EntertainmentSmall,HomeAudio,VoiceRecorder,10500,5
1647203,VOREYV2HWZDUFZ9Y,2015-09-06 22:14:34,2015,9,3381987673542980.000,300037347712641984.000,4299.000,1,NaN,NaN,COD,7,6.09775807960118E+018,-5.15252222605545E+018,EntertainmentSmall,HomeAudio,VoiceRecorder,10500,5
1647204,VOREYV2HWZDUFZ9Y,2015-09-11 16:11:52,2015,9,2388231540197500.000,300037347714801984.000,4599.000,1,NaN,NaN,COD,7,2.83122075392855E+018,3.60782023838401E+018,EntertainmentSmall,HomeAudio,VoiceRecorder,10500,5


In [27]:
len(consumer[consumer.duplicated(['fsn_id','order_date','order_id','order_item_id',
                              'gmv','cust_id','pincode',
                              'product_analytic_category','product_analytic_sub_category',
                             'product_analytic_vertical'])])

37355

In [28]:
#Removing duplicated values
consumer = consumer[~consumer.duplicated(['fsn_id','order_date','order_id','order_item_id',
                              'gmv','cust_id','pincode',
                              'product_analytic_category','product_analytic_sub_category',
                             'product_analytic_vertical'])]

In [29]:
consumer.loc[consumer.duplicated()]

,fsn_id,order_date,Year,Month,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,s1_fact.order_payment_type,sla,cust_id,pincode,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp,product_procurement_sla


In [30]:
#Checking nulls in gmv value
consumer.gmv.isnull().sum()

0

In [31]:
consumer.shape

(527240, 19)

### 3. deliverybdays and deliverycdays

In [32]:
# The columns deliverybdays and deliverycdays are populated with \N, which is incorrect.
# Let's replace them with null.
print(consumer.deliverybdays.value_counts().head())
print(consumer.deliverycdays.value_counts().head())

4    23535
5    23058
3    18708
6    17915
2    10924
Name: deliverybdays, dtype: int64
5    21584
6    19605
4    18637
7    14137
3    13212
Name: deliverycdays, dtype: int64


In [33]:
print(consumer.deliverybdays.isnull().sum()/len(consumer))
print(consumer.deliverycdays.isnull().sum()/len(consumer))

0.7776837872695547
0.7776837872695547


In [34]:
# We can drop delivercdays and deliverybdays column as it has 79% null values.
consumer.drop(['deliverybdays', 'deliverycdays'],1, inplace = True)

### 4. order_date

In [35]:
# Befor dealing with null values, let's first correct the data type of order_date
consumer['order_date'] = pd.to_datetime(consumer['order_date'])

In [36]:
# We now need to check if the dates are not outside July 2015 and June 2016.
consumer.loc[(consumer.order_date < '2015-07-01') | (consumer.order_date >= '2016-07-01')]

,fsn_id,order_date,Year,Month,order_id,order_item_id,gmv,units,s1_fact.order_payment_type,sla,cust_id,pincode,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp,product_procurement_sla
1071934,ACCCX3SGRHSRGTHZ,2016-07-01 01:07:36,2016,7,1611380362751500.000,1611380362751500.000,9999.000,1,COD,6,-6062166819020010496.000,8011106654138590208.000,CameraAccessory,CameraAccessory,Lens,19500,2
1074944,ACCD6HEGVCDHXPNQ,2016-07-03 16:12:47,2016,7,2602425887006100.000,2602425887006100.000,399.000,1,COD,6,2872406203167170048.000,-7543985763413010432.000,GamingHardware,GamingAccessory,GamePad,449,-1
1077916,ACCDFZEJAXVJDGS7,2016-07-01 01:03:17,2016,7,4611922526101700.000,4611922526101700.000,3699.000,1,Prepaid,2,-2126839628533280000.000,-3973362991373910016.000,EntertainmentSmall,HomeAudio,HomeAudioSpeaker,5290,-1
1080123,ACCDHQ8N6DGBYGKZ,2016-07-02 07:04:29,2016,7,3612045698494600.000,3612045698494600.000,949.000,1,COD,1,1164936338609760000.000,2370358404467279872.000,CameraAccessory,CameraAccessory,CameraTripod,1499,2
1082641,ACCDPYBFTPE5PRGT,2016-07-02 06:49:00,2016,7,4609992565256700.000,4609992565256700.000,1050.000,1,Prepaid,1,-9069302869700119552.000,4705607051154690048.000,GamingHardware,GamingAccessory,GamingHeadset,1792,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343782,TESE8HQPC9KY7EDY,2016-07-05 12:05:06,2016,7,1640574293935700.000,1640574293935700.000,2100.000,1,COD,7,-2.65648840718291E+018,-6.95142635777575E+018,CameraAccessory,CameraAccessory,Telescope,3000,1
1348417,VORDSAYVZVHS2THQ,2016-07-01 18:58:15,2016,7,3633598266571100.000,3633598266571100.000,4274.000,1,COD,4,-3.62228587849267E+018,1.70212685331306E+018,EntertainmentSmall,HomeAudio,VoiceRecorder,6359,2
1376093,ACCDVZ83VBHCWFFA,2015-05-19 13:42:09,2015,5,146499482.000,188485029.000,3149.000,1,COD,6,-6.14850311678963E+018,-3.21775258155408E+018,EntertainmentSmall,HomeAudio,HomeAudioSpeaker,6990,4
1402659,ACCE8G8BFKZRU3MW,2015-06-26 17:33:10,2015,6,163411313.000,207714163.000,2800.000,1,COD,11,5.80459991910742E+018,3.91454170274896E+018,EntertainmentSmall,HomeAudio,HomeAudioSpeaker,3000,3


- There is 608 records that lie outside the range. Let's delete those rows.

In [37]:
consumer = consumer.loc[(consumer.order_date >= '2015-07-01')]
consumer = consumer.loc[(consumer.order_date < '2016-07-01')]

### 5. s1_fact.order_payment_type

In [38]:
#Changing the name of the column s1_fact.order_payment_type
consumer.rename(columns={'s1_fact.order_payment_type':'order_payment_type'}, inplace=True)

In [39]:
consumer.order_payment_type.value_counts()

COD        384484
Prepaid    142556
Name: order_payment_type, dtype: int64

Clearly COD is preferred more than Prepaid order payment type.

### 6. pincode, custid

In [40]:
#Converting the datatype
consumer['pincode'] = pd.to_numeric(consumer['pincode'])

In [41]:
#Let's see the values of pincode field
consumer.pincode.min()

-9.22183035419401e+18

In [42]:
consumer.pincode.isnull().sum()

0

In [43]:
# Before handling null values, there are negative values for pincode which we need to handle.
# Let's make all the negative values as positive.
consumer.pincode = consumer.pincode.abs()

In [44]:
# Let's now check the frequency of pincodes to decide whether we can impute the missing pincodes with the highest frequency one.
consumer.pincode.value_counts()

9031043207091489792.000    4163
977901032130400000.000     3464
8649648456016409600.000    2194
6513330541343979520.000    2046
7346497166693510144.000    2020
                           ... 
6540960272762399744.000       1
6832281080269299712.000       1
3316478155595899904.000       1
2365466593809789952.000       1
559832405146460992.000        1
Name: pincode, Length: 6633, dtype: int64

In [45]:
# pincode and cust_id doesn't seem to be of any use

consumer.drop(['cust_id','pincode'], axis = 1, inplace = True)

### 7. product_mrp

In [46]:
consumer[(consumer.product_mrp == 0)].head()

,fsn_id,order_date,Year,Month,order_id,order_item_id,gmv,units,order_payment_type,sla,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp,product_procurement_sla
12807,ACCDGXGZQUGHFYHT,2015-10-23 19:26:01,2015,10,1424634732111000.000,1424634732111000.000,3500.000,1,COD,5,GamingHardware,GamingAccessory,GamingAccessoryKit,0,-1
15031,ACCDHHZZGFBDPJRP,2015-10-20 21:27:29,2015,10,3422117911872300.000,3422117911872300.000,690.000,1,Prepaid,6,CameraAccessory,CameraAccessory,CameraBattery,0,-1
15032,ACCDHHZZGFBDPJRP,2015-10-28 23:49:41,2015,10,2429116502765100.000,2429116502765100.000,690.000,1,COD,8,CameraAccessory,CameraAccessory,CameraBattery,0,-1
15529,ACCDHTZ4CZCECAFF,2015-10-27 01:05:20,2015,10,1427429193266100.000,1427429193266100.000,1050.000,1,COD,7,CameraAccessory,CameraAccessory,CameraBattery,0,-1
16980,ACCDK94FSKCZBJCU,2015-10-28 18:36:07,2015,10,4428925467622000.000,4428925467622000.000,1099.000,1,Prepaid,6,EntertainmentSmall,HomeAudio,Dock,0,-1


In [47]:
len(consumer[(consumer.product_mrp == 0)])

3236

In [48]:
#Removing values with 0 MRP, since that is not possible at all
consumer = consumer.loc[~(consumer.product_mrp==0)]

In [49]:
consumer['gmv_per_unit'] = consumer.gmv/consumer.units

In [50]:
#Replacing the values of MRP with GMV per unit where the values of GMV/unit is greater than MRP
consumer['product_mrp'].loc[consumer.gmv_per_unit>consumer.product_mrp] = consumer['gmv_per_unit']

In [51]:
consumer.loc[consumer.gmv_per_unit>consumer.product_mrp]

,fsn_id,order_date,Year,Month,order_id,order_item_id,gmv,units,order_payment_type,sla,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp,product_procurement_sla,gmv_per_unit


In [52]:
consumer.drop(['gmv_per_unit'],1,inplace=True)

### 8. sla and product_procurement_sla

In [53]:
consumer.shape

(523804, 15)

In [54]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,1)
sns.boxplot(y=consumer.sla, palette=("cubehelix"))

plt.subplot(1,2,2)
sns.boxplot(y=consumer.product_procurement_sla, palette=("cubehelix"))

<AxesSubplot:ylabel='product_procurement_sla'>

In [55]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,1)
sns.distplot(consumer.sla)

plt.subplot(1,2,2)
sns.distplot(consumer.product_procurement_sla)

<AxesSubplot:xlabel='product_procurement_sla', ylabel='Density'>

In [56]:
consumer.sla.describe(percentiles=[0.0,0.25,0.5,0.75,0.9,0.95,0.99,1.0])

count   523804.000
mean         5.885
std          2.697
min          0.000
0%           0.000
25%          4.000
50%          6.000
75%          7.000
90%          9.000
95%         11.000
99%         14.000
100%        60.000
max         60.000
Name: sla, dtype: float64

In [57]:
consumer.product_procurement_sla.describe(percentiles=[0.0,0.25,0.5,0.75,0.9,0.95,0.99,1.0])

count   523804.000
mean         2.560
std          1.715
min         -1.000
0%          -1.000
25%          2.000
50%          2.000
75%          3.000
90%          5.000
95%          5.000
99%          6.000
100%        15.000
max         15.000
Name: product_procurement_sla, dtype: float64

In [58]:
#Converting negative values to the positive
len(consumer.loc[consumer.product_procurement_sla<0])

10901

In [59]:
consumer.product_procurement_sla = abs(consumer.product_procurement_sla)

In [60]:
consumer.sla.std()

2.6969113975705605

In [61]:
#Taking three sigma values for outliers treatment
print(consumer.sla.mean()+(3*(consumer.sla.std())))
print(consumer.sla.mean()-(3*(consumer.sla.std())))

13.975410522025701
-2.206057863397664


In [62]:
consumer.product_procurement_sla.std()

1.6510180203178295

In [63]:
#Taking three sigma values for outliers treatment
print(consumer.product_procurement_sla.mean()+(3*(consumer.product_procurement_sla.std())))
print(consumer.product_procurement_sla.mean()-(3*(consumer.product_procurement_sla.std())))

7.554523312811053
-2.3515848090959235


In [64]:
# Capping the values at three sigma value
len(consumer[consumer.sla > 14])

3382

In [65]:
# Let's cap the SLAs.
consumer.loc[consumer.sla > 14,'sla'] = 14

In [66]:
# Similarly, the min value of product procurement sla is 0 and the max value is 15. However, three sigma value is 7. 
print(len(consumer[consumer.product_procurement_sla > 7]))

3813


In [67]:
# Let's cap the product procuremtn SLAs.
consumer.loc[consumer.product_procurement_sla > 7,'product_procurement_sla'] = 7

In [68]:
consumer.shape

(523804, 15)

In [69]:
consumer.loc[consumer.duplicated()]

,fsn_id,order_date,Year,Month,order_id,order_item_id,gmv,units,order_payment_type,sla,product_analytic_category,product_analytic_sub_category,product_analytic_vertical,product_mrp,product_procurement_sla


### Duplicates removal

In [70]:
len(consumer[consumer.duplicated(['order_id','order_item_id'])])

12587

- Clearly, there can't be two orders with the same combination of order id and order item id that were ordered at the same timestamp.
- We can hence, drop the duplicates.

In [71]:
consumer = consumer[~consumer.duplicated(['order_id','order_item_id'])]

In [72]:
consumer.describe()

,Year,Month,order_id,order_item_id,gmv,units,sla,product_mrp,product_procurement_sla
count,511217.000,511217.000,511217.000,511217.000,511217.000,511217.000,511217.000,511217.000,511217.000
mean,2015.576,6.228,2835122101205655.000,9390790484550196.000,1290.685,1.024,5.862,2500.782,2.555
std,0.494,3.674,1290983206428331.750,43696556291619304.000,2438.475,0.239,2.621,4699.526,1.414
min,2015.000,1.000,162957597.000,207200079.000,0.000,1.000,0.000,79.000,0.000
25%,2015.000,3.000,1595735407225200.000,1598206083624100.000,299.000,1.000,4.000,766.000,2.000
50%,2016.000,6.000,2609704882795000.000,2613097725117400.000,550.000,1.000,6.000,1200.000,2.000
75%,2016.000,10.000,3615581804522700.000,4388978807078500.000,1500.000,1.000,7.000,2800.000,3.000
max,2016.000,12.000,4641664595370500.000,300037347717704000.000,148050.000,39.000,14.000,180000.000,7.000


In [73]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,1)
sns.distplot(consumer.gmv)

plt.subplot(1,2,2)
sns.distplot(consumer.product_mrp)

plt.show()

### 3. Feature Engineering and KPI
- Pricing KPI
- Listed Price

In [74]:
#2. gmv (Gross Merchendising Value - The cost price at which the item is sold multiplied by number of units)

# Let's derive listing price, which is nothing but gmv/units

consumer['listing_price'] = round((consumer.gmv/consumer.units),2)

In [75]:
#Let's check if there are any rows with listing price > MRP

len(consumer.loc[consumer.listing_price>consumer.product_mrp])

0

### Discount and Promotion related KPI
#### Dicount

In [76]:
# Let's now calculate the discount %, which is nothing but (mrp-list price)/mrp
consumer['discount'] = round(((consumer.product_mrp - consumer.listing_price)/(consumer.product_mrp)),2)

In [77]:
consumer['discount'].describe()

count   511217.000
mean         0.452
std          0.245
min          0.000
25%          0.260
50%          0.460
75%          0.630
max          1.000
Name: discount, dtype: float64

#### Total Price

In [78]:
consumer['Order_Item_Value'] = consumer['product_mrp'] * consumer['units']

#### Mapping Week into the Data

In [79]:
# We can create the week number
consumer['week'] = np.where(consumer.Year == 2015, (consumer.order_date.dt.week - pd.to_datetime('2015-07-01').week + 1), consumer.order_date.dt.week+27)

# Dates like 2016-01-01 will be 53rd week as per ISO standard, hence the week value would be 53+27=80.
# We can make those values as week 27
consumer.week.values[(consumer.Year == 2016) & (consumer.week == 80)] = 27

### Product assortment and quality related KPI
#### Payment type

In [80]:
### Prepaid = '1' or COD = '0'
consumer['order_payment_type'] = np.where(consumer['order_payment_type'] == "Prepaid",1,0)

### Seasonality and Trend related KPI
#### Calendar

In [81]:
### Creating Calendar for the period
calendar = pd.DataFrame(pd.date_range('2015-07-01','2016-06-30').tolist(), columns = ['Date'])
### Mapping week in the calendar
calendar['week'] = calendar.Date.dt.week
### Jan 2016 should be week 54 ,not week 1.
calendar['week'] = np.where((calendar['week'] <= 26) & (calendar.Date.dt.year == 2016), calendar['week']+53, calendar['week'])

#### Special Sale

In [82]:
### Special Sales List

special_sales_list = ["2015-07-18","2015-07-19","2015-08-15","2015-08-16","2015-08-17","2015-08-28","2015-08-29",
                      "2015-08-30","2015-10-15","2015-10-16","2015-10-17","2015-11-07","2015-11-08","2015-11-09",
                      "2015-11-10","2015-11-11","2015-11-12","2015-11-13","2015-11-14","2015-12-25","2015-12-26",
                      "2015-12-27","2015-12-28","2015-12-29","2015-12-30","2015-12-31","2016-01-01","2016-01-02",
                      "2016-01-03","2016-01-20","2016-01-21","2016-01-22","2016-02-01","2016-02-02","2016-02-14",
                      "2016-02-15","2016-02-20","2016-02-21","2016-03-07","2016-03-08","2016-03-09","2016-05-25",
                      "2016-05-26","2016-05-27"]

ss_list = pd.DataFrame(special_sales_list,columns = ['Date'])
ss_list['Date'] = pd.to_datetime(ss_list['Date'])
ss_list['Special_sales'] = True

In [83]:
calendar = calendar.merge(ss_list, 'left')
calendar.fillna(False, inplace = True)

In [84]:
calendar['Special_sales'] = calendar['Special_sales'].astype(int)

In [85]:
calendar.head()

,Date,week,Special_sales
0,2015-07-01,27,0
1,2015-07-02,27,0
2,2015-07-03,27,0
3,2015-07-04,27,0
4,2015-07-05,27,0


### Payday

In [86]:
calendar['Payday'] = ((calendar['Date'].dt.day == 1) | (calendar['Date'].dt.day == 15)).astype(int)

#### Climate Data

In [87]:
### Ontario Climate data of year 2015-2016 
ontario_climate_2015 = pd.DataFrame(pd.read_csv('ONTARIO-2015.csv',encoding="ISO-8859-1",skiprows=24))
ontario_climate_2016 = pd.DataFrame(pd.read_csv('ONTARIO-2016.csv',encoding="ISO-8859-1",skiprows=24))

In [88]:
### Merge Calendar with dataset on week

ontario_climate = ontario_climate_2015.append(ontario_climate_2016)
ontario_climate = ontario_climate.reset_index()
ontario_climate.head()

,index,Date/Time,Year,Month,Day,Data Quality,Max Temp (Â°C),Max Temp Flag,Min Temp (Â°C),Min Temp Flag,...,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
0,0,2015-01-01,2015,1,1,â,0.000,NaN,-8.500,NaN,...,0.000,NaN,0.000,NaN,0.000,NaN,NaN,NaN,NaN,NaN
1,1,2015-01-02,2015,1,2,â,3.000,NaN,-3.000,NaN,...,0.000,NaN,0.000,NaN,0.000,NaN,NaN,NaN,NaN,NaN
2,2,2015-01-03,2015,1,3,â,2.500,NaN,-4.000,NaN,...,0.000,NaN,24.000,NaN,0.000,NaN,NaN,NaN,NaN,NaN
3,3,2015-01-04,2015,1,4,â,2.500,NaN,0.000,NaN,...,1.000,NaN,1.000,NaN,0.000,NaN,NaN,NaN,NaN,NaN
4,4,2015-01-05,2015,1,5,â,-10.000,NaN,-13.500,NaN,...,3.000,NaN,3.000,NaN,1.000,NaN,NaN,NaN,NaN,NaN


In [89]:
### Checking for any nan values

round((ontario_climate.isnull().sum()/len(ontario_climate.index))*100,2)

index                         0.000
Date/Time                     0.000
Year                          0.000
Month                         0.000
Day                           0.000
Data Quality                 37.620
Max Temp (Â°C)               39.530
Max Temp Flag                98.080
Min Temp (Â°C)               44.050
Min Temp Flag                93.570
Mean Temp (Â°C)              44.050
Mean Temp Flag               93.570
Heat Deg Days (Â°C)          44.050
Heat Deg Days Flag           93.570
Cool Deg Days (Â°C)          44.050
Cool Deg Days Flag           93.570
Total Rain (mm)              37.620
Total Rain Flag              98.630
Total Snow (cm)              37.620
Total Snow Flag              97.810
Total Precip (mm)            37.620
Total Precip Flag            97.400
Snow on Grnd (cm)            37.620
Snow on Grnd Flag            98.910
Dir of Max Gust (10s deg)   100.000
Dir of Max Gust Flag        100.000
Spd of Max Gust (km/h)      100.000
Spd of Max Gust Flag        

In [90]:
### Dropping columns we do not require in the analysis.
ontario_climate.drop(['index','Data Quality','Max Temp Flag','Min Temp Flag','Mean Temp Flag',
                      'Heat Deg Days Flag','Cool Deg Days Flag','Total Rain Flag','Total Snow Flag',
                      'Total Precip Flag','Snow on Grnd Flag','Dir of Max Gust (10s deg)','Dir of Max Gust Flag',
                      'Spd of Max Gust (km/h)','Spd of Max Gust Flag'], axis = 1, inplace = True)

In [91]:
ontario_climate.columns = ['Date','Year','Month','Day','max_temp_C','min_temp_C','mean_temp_C','heat_deg_days',
                           'cool_deg_days','total_rain_mm','total_snow_cm','total_precip_mm','snow_on_grnd_cm']

In [92]:
ontario_climate['Date'] = ontario_climate['Date'].apply(pd.to_datetime)

In [93]:
### Keeping Climate data from July 15 to June 16

ontario_climate=ontario_climate[(ontario_climate['Month'] >= 7) & (ontario_climate['Year'] == 2015) 
                               |(ontario_climate['Month'] <= 6) & (ontario_climate['Year'] == 2016)]

In [94]:
### Mapping week in the Climate data
ontario_climate['week'] = ontario_climate.Date.dt.week

### Jan 2016 should be week 54 ,not week 1.
ontario_climate['week'] = np.where((ontario_climate['week'] <= 26) & (ontario_climate['Year'] == 2016), ontario_climate['week']+53, ontario_climate['week'])

ontario_climate = ontario_climate.reset_index()
ontario_climate.drop('index',axis=1,inplace=True)
ontario_climate.head()

,Date,Year,Month,Day,max_temp_C,min_temp_C,mean_temp_C,heat_deg_days,cool_deg_days,total_rain_mm,total_snow_cm,total_precip_mm,snow_on_grnd_cm,week
0,2015-07-01,2015,7,1,24.500,17.000,20.800,0.000,2.800,0.000,0.000,0.000,0.000,27
1,2015-07-02,2015,7,2,24.000,14.000,19.000,0.000,1.000,0.000,0.000,0.000,0.000,27
2,2015-07-03,2015,7,3,25.000,10.000,17.500,0.500,0.000,0.000,0.000,0.000,0.000,27
3,2015-07-04,2015,7,4,26.000,11.000,18.500,0.000,0.500,0.000,0.000,0.000,0.000,27
4,2015-07-05,2015,7,5,28.000,14.000,21.000,0.000,3.000,0.000,0.000,0.000,0.000,27


In [95]:
### Checking for any nan values

round((ontario_climate.isnull().sum()/len(ontario_climate.index))*100,2)

Date               0.000
Year               0.000
Month              0.000
Day                0.000
max_temp_C        43.170
min_temp_C        47.810
mean_temp_C       47.810
heat_deg_days     47.810
cool_deg_days     47.810
total_rain_mm     41.260
total_snow_cm     41.260
total_precip_mm   41.260
snow_on_grnd_cm   41.260
week               0.000
dtype: float64

In [96]:
### Replacing Nan with mean value
ontario_climate['max_temp_C'] = ontario_climate['max_temp_C'].fillna(ontario_climate['max_temp_C'].mean())
ontario_climate['min_temp_C'] = ontario_climate['min_temp_C'].fillna(ontario_climate['min_temp_C'].mean())
ontario_climate['mean_temp_C'] = ontario_climate['mean_temp_C'].fillna(ontario_climate['mean_temp_C'].mean())
ontario_climate['heat_deg_days'] = ontario_climate['heat_deg_days'].fillna(ontario_climate['heat_deg_days'].mean())
ontario_climate['cool_deg_days'] = ontario_climate['cool_deg_days'].fillna(ontario_climate['cool_deg_days'].mean())
ontario_climate['total_rain_mm'] = ontario_climate['total_rain_mm'].fillna(ontario_climate['total_rain_mm'].mean())
ontario_climate['total_snow_cm'] = ontario_climate['total_snow_cm'].fillna(ontario_climate['total_snow_cm'].mean())
ontario_climate['total_precip_mm'] = ontario_climate['total_precip_mm'].fillna(ontario_climate['total_precip_mm'].mean())
ontario_climate['snow_on_grnd_cm'] = ontario_climate['snow_on_grnd_cm'].fillna(ontario_climate['snow_on_grnd_cm'].mean())

In [97]:
ontario_climate.head()

,Date,Year,Month,Day,max_temp_C,min_temp_C,mean_temp_C,heat_deg_days,cool_deg_days,total_rain_mm,total_snow_cm,total_precip_mm,snow_on_grnd_cm,week
0,2015-07-01,2015,7,1,24.500,17.000,20.800,0.000,2.800,0.000,0.000,0.000,0.000,27
1,2015-07-02,2015,7,2,24.000,14.000,19.000,0.000,1.000,0.000,0.000,0.000,0.000,27
2,2015-07-03,2015,7,3,25.000,10.000,17.500,0.500,0.000,0.000,0.000,0.000,0.000,27
3,2015-07-04,2015,7,4,26.000,11.000,18.500,0.000,0.500,0.000,0.000,0.000,0.000,27
4,2015-07-05,2015,7,5,28.000,14.000,21.000,0.000,3.000,0.000,0.000,0.000,0.000,27


### Other KPI
Net Promoters Score & Stock_Index

In [98]:
nps_score = pd.read_excel("Media data and other information.xlsx", sheet_name='Monthly NPS Score', skiprows=1)

In [99]:
### Transforming NPS and Stock_index
nps_score = nps_score.T.reset_index(drop=True)
nps_score.columns = ['NPS','Stock_Index']
nps_score = nps_score.drop(nps_score.index[[0]]).reset_index(drop=True)

In [100]:
### Adding Month and Year
nps_score['Month'] = pd.Series([7,8,9,10,11,12,1,2,3,4,5,6])
nps_score['Year'] = pd.Series([2015,2015,2015,2015,2015,2015,2016,2016,2016,2016,2016,2016])

In [101]:
nps_score['NPS'] = nps_score['NPS'].astype(float)
nps_score['Stock_Index'] = nps_score['Stock_Index'].astype(float)

In [102]:
nps_score.head()

,NPS,Stock_Index,Month,Year
0,54.600,1177.000,7,2015
1,59.987,1206.000,8,2015
2,46.925,1101.000,9,2015
3,44.398,1210.000,10,2015
4,47.000,1233.000,11,2015


### Mapping KPI on Calendar

In [103]:
calendar = calendar.merge(ontario_climate, 'left')

In [104]:
calendar = calendar.merge(nps_score, 'left')

In [105]:
# We can create the week number
calendar['week'] = np.where(calendar.Date.dt.year == 2015, (calendar.Date.dt.week - pd.to_datetime('2015-07-01').week + 1), calendar.Date.dt.week+27)

# Dates like 2016-01-01 will be 53rd week as per ISO standard, hence the week value would be 53+27=80.
# We can make those values as week 27
calendar.week.values[(calendar.Date.dt.year == 2016) & (calendar.week == 80)] = 27

In [106]:
calendar.head()

,Date,week,Special_sales,Payday,Year,Month,Day,max_temp_C,min_temp_C,mean_temp_C,heat_deg_days,cool_deg_days,total_rain_mm,total_snow_cm,total_precip_mm,snow_on_grnd_cm,NPS,Stock_Index
0,2015-07-01,1,0,1,2015,7,1,24.500,17.000,20.800,0.000,2.800,0.000,0.000,0.000,0.000,54.600,1177.000
1,2015-07-02,1,0,0,2015,7,2,24.000,14.000,19.000,0.000,1.000,0.000,0.000,0.000,0.000,54.600,1177.000
2,2015-07-03,1,0,0,2015,7,3,25.000,10.000,17.500,0.500,0.000,0.000,0.000,0.000,0.000,54.600,1177.000
3,2015-07-04,1,0,0,2015,7,4,26.000,11.000,18.500,0.000,0.500,0.000,0.000,0.000,0.000,54.600,1177.000
4,2015-07-05,1,0,0,2015,7,5,28.000,14.000,21.000,0.000,3.000,0.000,0.000,0.000,0.000,54.600,1177.000


In [107]:
calendar = pd.DataFrame(calendar.groupby('week').agg({'NPS':'mean','Stock_Index':'mean',
                                                             'Special_sales':'mean','Payday':'mean',
                                                             'max_temp_C':'mean','min_temp_C':'mean',
                                                             'mean_temp_C':'mean','heat_deg_days':'mean',
                                                             'cool_deg_days':'mean','total_rain_mm':'mean',
                                                             'total_snow_cm':'mean','total_precip_mm':'mean',
                                                             'snow_on_grnd_cm':'mean'}))

In [108]:
calendar.reset_index(inplace = True)

In [109]:
calendar.head()

,week,NPS,Stock_Index,Special_sales,Payday,max_temp_C,min_temp_C,mean_temp_C,heat_deg_days,cool_deg_days,total_rain_mm,total_snow_cm,total_precip_mm,snow_on_grnd_cm
0,1,54.600,1177.000,0.000,0.200,25.500,13.200,19.360,0.100,1.460,0.000,0.000,0.000,0.000
1,2,54.600,1177.000,0.000,0.000,24.025,14.385,19.263,1.059,2.322,4.155,0.029,4.184,0.041
2,3,54.600,1177.000,0.286,0.143,27.383,16.314,21.906,0.816,4.722,1.569,0.029,1.598,0.041
3,4,54.600,1177.000,0.000,0.000,28.240,14.342,20.540,1.632,4.172,1.141,0.029,1.170,0.041
4,5,56.139,1185.286,0.000,0.143,29.811,16.242,23.092,0.816,5.908,4.341,0.029,4.370,0.041


### Advertisement Related KPI
Marketing

In [110]:
### Marketing Investment Data
marketing = pd.read_excel("Media data and other information.xlsx", sheet_name='Media Investment', skipfooter = 4, skiprows=2)

In [111]:
marketing.drop('Unnamed: 0', axis = 1, inplace = True)
marketing.replace(np.nan,0,inplace = True)
marketing['Date'] = pd.to_datetime(marketing[['Year', 'Month']].assign(DAY=1))
marketing.set_index('Date', inplace = True)
marketing

,Year,Month,Total Investment,TV,Digital,Sponsorship,Content Marketing,Online marketing,Affiliates,SEM,Radio,Other
Date,,,,,,,,,,,,
2015-07-01,2015.000,7.000,17.062,0.215,2.533,7.414,0.001,1.327,0.547,5.024,0.000,0.000
2015-08-01,2015.000,8.000,5.064,0.006,1.278,1.063,0.000,0.129,0.074,2.514,0.000,0.000
2015-09-01,2015.000,9.000,96.254,3.880,1.357,62.788,0.610,16.380,5.038,6.202,0.000,0.000
2015-10-01,2015.000,10.000,170.156,6.145,12.622,84.673,3.444,24.372,6.974,31.927,0.000,0.000
2015-11-01,2015.000,11.000,51.216,4.221,1.275,14.172,0.169,19.562,6.596,5.222,0.000,0.000
2015-12-01,2015.000,12.000,106.745,5.398,3.063,56.705,1.067,22.504,6.827,11.181,0.000,0.000
2016-01-01,2016.000,1.000,74.196,4.380,0.456,4.200,0.900,22.900,7.370,4.200,2.700,27.090
2016-02-01,2016.000,2.000,48.052,2.586,1.913,11.726,0.597,19.892,6.465,4.874,0.000,0.000


In [112]:
### Renaming the columns

marketing.columns = ['Year','Month','Total_Investment','TV','Digital','Sponsorship','Content_marketing',
                     'Online_marketing','Affiliates','SEM','Radio','Other']

In [113]:
### convert to datetimeindex
marketing.index = pd.to_datetime(marketing.index)

In [114]:
marketing

,Year,Month,Total_Investment,TV,Digital,Sponsorship,Content_marketing,Online_marketing,Affiliates,SEM,Radio,Other
Date,,,,,,,,,,,,
2015-07-01,2015.000,7.000,17.062,0.215,2.533,7.414,0.001,1.327,0.547,5.024,0.000,0.000
2015-08-01,2015.000,8.000,5.064,0.006,1.278,1.063,0.000,0.129,0.074,2.514,0.000,0.000
2015-09-01,2015.000,9.000,96.254,3.880,1.357,62.788,0.610,16.380,5.038,6.202,0.000,0.000
2015-10-01,2015.000,10.000,170.156,6.145,12.622,84.673,3.444,24.372,6.974,31.927,0.000,0.000
2015-11-01,2015.000,11.000,51.216,4.221,1.275,14.172,0.169,19.562,6.596,5.222,0.000,0.000
2015-12-01,2015.000,12.000,106.745,5.398,3.063,56.705,1.067,22.504,6.827,11.181,0.000,0.000
2016-01-01,2016.000,1.000,74.196,4.380,0.456,4.200,0.900,22.900,7.370,4.200,2.700,27.090
2016-02-01,2016.000,2.000,48.052,2.586,1.913,11.726,0.597,19.892,6.465,4.874,0.000,0.000


In [115]:
### add new next month for correct resample
idx = marketing.index[-1] + pd.offsets.MonthBegin(1)
idx

Timestamp('2016-03-01 00:00:00')

In [116]:
marketing = marketing.append(marketing.iloc[[-1]].rename({marketing.index[-1]: idx}))
marketing

,Year,Month,Total_Investment,TV,Digital,Sponsorship,Content_marketing,Online_marketing,Affiliates,SEM,Radio,Other
Date,,,,,,,,,,,,
2015-07-01,2015.000,7.000,17.062,0.215,2.533,7.414,0.001,1.327,0.547,5.024,0.000,0.000
2015-08-01,2015.000,8.000,5.064,0.006,1.278,1.063,0.000,0.129,0.074,2.514,0.000,0.000
2015-09-01,2015.000,9.000,96.254,3.880,1.357,62.788,0.610,16.380,5.038,6.202,0.000,0.000
2015-10-01,2015.000,10.000,170.156,6.145,12.622,84.673,3.444,24.372,6.974,31.927,0.000,0.000
2015-11-01,2015.000,11.000,51.216,4.221,1.275,14.172,0.169,19.562,6.596,5.222,0.000,0.000
2015-12-01,2015.000,12.000,106.745,5.398,3.063,56.705,1.067,22.504,6.827,11.181,0.000,0.000
2016-01-01,2016.000,1.000,74.196,4.380,0.456,4.200,0.900,22.900,7.370,4.200,2.700,27.090
2016-02-01,2016.000,2.000,48.052,2.586,1.913,11.726,0.597,19.892,6.465,4.874,0.000,0.000
2016-03-01,2016.000,2.000,48.052,2.586,1.913,11.726,0.597,19.892,6.465,4.874,0.000,0.000


In [117]:
#Resampling the data on weekly frequency
marketing = marketing.resample('W').ffill().iloc[:-1]
marketing

,Year,Month,Total_Investment,TV,Digital,Sponsorship,Content_marketing,Online_marketing,Affiliates,SEM,Radio,Other
Date,,,,,,,,,,,,
2015-07-05,2015.000,7.000,17.062,0.215,2.533,7.414,0.001,1.327,0.547,5.024,0.000,0.000
2015-07-12,2015.000,7.000,17.062,0.215,2.533,7.414,0.001,1.327,0.547,5.024,0.000,0.000
2015-07-19,2015.000,7.000,17.062,0.215,2.533,7.414,0.001,1.327,0.547,5.024,0.000,0.000
2015-07-26,2015.000,7.000,17.062,0.215,2.533,7.414,0.001,1.327,0.547,5.024,0.000,0.000
2015-08-02,2015.000,8.000,5.064,0.006,1.278,1.063,0.000,0.129,0.074,2.514,0.000,0.000
2015-08-09,2015.000,8.000,5.064,0.006,1.278,1.063,0.000,0.129,0.074,2.514,0.000,0.000
2015-08-16,2015.000,8.000,5.064,0.006,1.278,1.063,0.000,0.129,0.074,2.514,0.000,0.000
2015-08-23,2015.000,8.000,5.064,0.006,1.278,1.063,0.000,0.129,0.074,2.514,0.000,0.000
2015-08-30,2015.000,8.000,5.064,0.006,1.278,1.063,0.000,0.129,0.074,2.514,0.000,0.000


In [118]:
### divide by size of months
marketing['Total_Investment'] /= marketing.resample('MS')['Total_Investment'].transform('size')
marketing['TV'] /= marketing.resample('MS')['TV'].transform('size')
marketing['Digital'] /= marketing.resample('MS')['Digital'].transform('size')
marketing['Sponsorship'] /= marketing.resample('MS')['Sponsorship'].transform('size')
marketing['Content_marketing'] /= marketing.resample('MS')['Content_marketing'].transform('size')
marketing['Online_marketing'] /= marketing.resample('MS')['Online_marketing'].transform('size')
marketing['Affiliates'] /= marketing.resample('MS')['Affiliates'].transform('size')
marketing['SEM'] /= marketing.resample('MS')['SEM'].transform('size')
marketing['Radio'] /= marketing.resample('MS')['Radio'].transform('size')
marketing['Other'] /= marketing.resample('MS')['Other'].transform('size')

In [119]:
marketing.head()

,Year,Month,Total_Investment,TV,Digital,Sponsorship,Content_marketing,Online_marketing,Affiliates,SEM,Radio,Other
Date,,,,,,,,,,,,
2015-07-05,2015.000,7.000,4.265,0.054,0.633,1.854,0.000,0.332,0.137,1.256,0.000,0.000
2015-07-12,2015.000,7.000,4.265,0.054,0.633,1.854,0.000,0.332,0.137,1.256,0.000,0.000
2015-07-19,2015.000,7.000,4.265,0.054,0.633,1.854,0.000,0.332,0.137,1.256,0.000,0.000
2015-07-26,2015.000,7.000,4.265,0.054,0.633,1.854,0.000,0.332,0.137,1.256,0.000,0.000
2015-08-02,2015.000,8.000,1.013,0.001,0.256,0.213,0.000,0.026,0.015,0.503,0.000,0.000


In [120]:
marketing.reset_index(inplace = True)

###  Mapping week in the marketing

marketing['Date'] = pd.to_datetime(marketing['Date'])
# We can create the week number
marketing['week'] = np.where(marketing.Date.dt.year == 2015, (marketing.Date.dt.week - pd.to_datetime('2015-07-01').week + 1), marketing.Date.dt.week+27)

marketing.week.values[(marketing.Date.dt.year == 2016) & (marketing.week == 80)] = 27
marketing.sort_values('week', inplace = True)

In [121]:
marketing.head()

,Date,Year,Month,Total_Investment,TV,Digital,Sponsorship,Content_marketing,Online_marketing,Affiliates,SEM,Radio,Other,week
0,2015-07-05,2015.000,7.000,4.265,0.054,0.633,1.854,0.000,0.332,0.137,1.256,0.000,0.000,1
1,2015-07-12,2015.000,7.000,4.265,0.054,0.633,1.854,0.000,0.332,0.137,1.256,0.000,0.000,2
2,2015-07-19,2015.000,7.000,4.265,0.054,0.633,1.854,0.000,0.332,0.137,1.256,0.000,0.000,3
3,2015-07-26,2015.000,7.000,4.265,0.054,0.633,1.854,0.000,0.332,0.137,1.256,0.000,0.000,4
4,2015-08-02,2015.000,8.000,1.013,0.001,0.256,0.213,0.000,0.026,0.015,0.503,0.000,0.000,5


### Adstock

In [122]:
def adstocked_advertising(adstock_rate=0.5, advertising = marketing):
    
    adstocked_advertising = []
    for i in range(len(advertising)):
        if i == 0: 
            adstocked_advertising.append(advertising.iloc[i])
        else:
            adstocked_advertising.append(advertising.iloc[i] + adstock_rate * advertising.iloc[i-1])            
    return adstocked_advertising

In [123]:
adstock = pd.DataFrame()

In [124]:
adstock['TV_ads'] = adstocked_advertising(adstock_rate=0.5, advertising = marketing['TV'])

adstock['Digital_ads'] = adstocked_advertising(adstock_rate=0.5, advertising = marketing['Digital'])

adstock['Sponsorship_ads'] = adstocked_advertising(adstock_rate=0.5, advertising = marketing['Sponsorship'])

adstock['Content_marketing_ads'] = adstocked_advertising(adstock_rate=0.5, advertising = marketing['Content_marketing'])

adstock['Online_marketing_ads'] = adstocked_advertising(adstock_rate=0.5, advertising = marketing['Online_marketing'])

adstock['Affiliates_ads'] = adstocked_advertising(adstock_rate=0.5, advertising = marketing['Affiliates'])

adstock['SEM_ads'] = adstocked_advertising(adstock_rate=0.5, advertising = marketing['SEM'])

adstock['Radio_ads'] = adstocked_advertising(adstock_rate=0.5, advertising = marketing['Radio'])

adstock['Other_ads'] = adstocked_advertising(adstock_rate=0.5, advertising = marketing['Other'])

In [125]:
adstock.head()

,TV_ads,Digital_ads,Sponsorship_ads,Content_marketing_ads,Online_marketing_ads,Affiliates_ads,SEM_ads,Radio_ads,Other_ads
0,0.054,0.633,1.854,0.000,0.332,0.137,1.256,0.000,0.000
1,0.081,0.950,2.780,0.000,0.498,0.205,1.884,0.000,0.000
2,0.081,0.950,2.780,0.000,0.498,0.205,1.884,0.000,0.000
3,0.081,0.950,2.780,0.000,0.498,0.205,1.884,0.000,0.000
4,0.028,0.572,1.139,0.000,0.192,0.083,1.131,0.000,0.000


### Mapping marketing and adstock

In [126]:
marketing = pd.concat([marketing,adstock] ,axis=1)

In [127]:
marketing.head()

,Date,Year,Month,Total_Investment,TV,Digital,Sponsorship,Content_marketing,Online_marketing,Affiliates,...,week,TV_ads,Digital_ads,Sponsorship_ads,Content_marketing_ads,Online_marketing_ads,Affiliates_ads,SEM_ads,Radio_ads,Other_ads
0,2015-07-05,2015.000,7.000,4.265,0.054,0.633,1.854,0.000,0.332,0.137,...,1,0.054,0.633,1.854,0.000,0.332,0.137,1.256,0.000,0.000
1,2015-07-12,2015.000,7.000,4.265,0.054,0.633,1.854,0.000,0.332,0.137,...,2,0.081,0.950,2.780,0.000,0.498,0.205,1.884,0.000,0.000
2,2015-07-19,2015.000,7.000,4.265,0.054,0.633,1.854,0.000,0.332,0.137,...,3,0.081,0.950,2.780,0.000,0.498,0.205,1.884,0.000,0.000
3,2015-07-26,2015.000,7.000,4.265,0.054,0.633,1.854,0.000,0.332,0.137,...,4,0.081,0.950,2.780,0.000,0.498,0.205,1.884,0.000,0.000
4,2015-08-02,2015.000,8.000,1.013,0.001,0.256,0.213,0.000,0.026,0.015,...,5,0.028,0.572,1.139,0.000,0.192,0.083,1.131,0.000,0.000


### Product premium-ness

In [128]:
# The premium-ness of the product depends on the MRP. Higher the MRP, more premium is the product.
# Let's check the percentiles of MRP in the dataset.

consumer.product_mrp.describe(percentiles=[0.25,0.5,0.75,0.8,0.9,0.95,0.99])

count   511217.000
mean      2500.782
std       4699.526
min         79.000
25%        766.000
50%       1200.000
75%       2800.000
80%       3150.000
90%       5399.000
95%       6990.000
99%      18999.000
max     180000.000
Name: product_mrp, dtype: float64

In [129]:
# Let's assume that products with MRP greater than 90 percentile to be premium products.
# Create a dataframe with mrp, number of units sold and gmv against each product vertical to analyse better.
prod_cat = pd.DataFrame(pd.pivot_table(consumer, values = ['units','product_mrp', 'gmv'], index = ['product_analytic_vertical'], 
               aggfunc={'units':np.sum, 'product_mrp':np.mean, 'gmv':np.sum}).to_records())

In [130]:
# Marking products with MRP greater than 90th percentile with 1 and rest with 0
prod_cat['premium_product'] = np.where((prod_cat.product_mrp>consumer.product_mrp.quantile(0.9)),1,0)

In [131]:
prod_cat.loc[prod_cat.premium_product==1]

,product_analytic_vertical,gmv,product_mrp,units,premium_product
10,CameraLEDLight,20215.000,6738.333,3,1
16,DJController,2335536.000,17692.871,191,1
39,KaraokePlayer,23500.000,12000.000,2,1
41,MotionController,4134009.000,5841.381,1057,1
43,SlingBox,20495.000,7999.000,5,1
45,SoundMixer,638073.000,12271.377,69,1
48,Teleconverter,15299.000,20490.000,1,1


- Clearly, Teleconverter, SoundMixer, SlingBox, MotionController, KaraokePlayer, DJController are premium products. All other products are mass products.
- Let's visualise how the premium products contribute towards the GMV.

In [132]:
plt.figure(figsize=(15,5))
sns.barplot(x = prod_cat.product_analytic_vertical, y=prod_cat.gmv, hue=prod_cat.premium_product)
plt.xticks(rotation=90)
plt.show()

- We can clearly see that, maximum revenue is generated through mass producs like HomeAudioSpeaker, Lens, GamingPad, etc and not premium products that contibute quite less towards revenue.
- The company hence should focus more on mass products than premium products.

In [133]:
consumer = consumer.merge(prod_cat[['product_analytic_vertical', 'premium_product']] , left_on='product_analytic_vertical', 
            right_on='product_analytic_vertical',
                   how = 'inner')

In [134]:
sales = consumer.copy()

In [135]:
consumer.drop(['product_analytic_vertical'],1,inplace=True)

In [136]:
consumer.head()

,fsn_id,order_date,Year,Month,order_id,order_item_id,gmv,units,order_payment_type,sla,product_analytic_category,product_analytic_sub_category,product_mrp,product_procurement_sla,listing_price,discount,Order_Item_Value,week,premium_product
0,ACCCX3S58G7B5F6P,2015-10-17 15:11:54,2015,10,3419300926147000.000,3419300926147000.000,6400.000,1,0,5,CameraAccessory,CameraAccessory,7190.000,0,6400.000,0.110,7190.000,16,0
1,ACCCX3S58G7B5F6P,2015-10-19 10:07:22,2015,10,1420830839915200.000,1420830839915200.000,6900.000,1,0,7,CameraAccessory,CameraAccessory,7190.000,0,6900.000,0.040,7190.000,17,0
2,ACCCX3S5AHMF55FV,2015-10-20 15:45:56,2015,10,2421912925714800.000,2421912925714800.000,1990.000,1,0,10,CameraAccessory,CameraAccessory,2099.000,3,1990.000,0.050,2099.000,17,0
3,ACCCX3S5AHMF55FV,2015-10-14 12:05:15,2015,10,4416592101738400.000,4416592101738400.000,1690.000,1,1,4,CameraAccessory,CameraAccessory,2099.000,3,1690.000,0.190,2099.000,16,0
4,ACCCX3S5AHMF55FV,2015-10-17 21:25:03,2015,10,4419525153426400.000,4419525153426400.000,1618.000,1,1,6,CameraAccessory,CameraAccessory,2099.000,3,1618.000,0.230,2099.000,16,0


# 4. Aggregation
## 1. Camera Sub Category

In [137]:
camera_df = consumer[consumer['product_analytic_sub_category'] == 'CameraAccessory']

 ### Removing outliers is important as
 - There may be some garbage value.
 - Bulk orders can skew the analysis

In [138]:
### Outlier Analysis
fig, axs = plt.subplots(1,3, figsize = (20,4))
plt1 = sns.boxplot(camera_df['gmv'], ax = axs[0])
plt2 = sns.boxplot(camera_df['units'], ax = axs[2])
plt4 = sns.boxplot(camera_df['product_mrp'], ax = axs[1])
plt.tight_layout()

In [139]:
### Treating outliers
### Outlier treatment for gmv & product_mrp
Q1 = camera_df.gmv.quantile(0.25)
Q3 = camera_df.gmv.quantile(0.75)
IQR = Q3 - Q1
camera_df = camera_df[(camera_df.gmv >= Q1 - 1.5*IQR) & (camera_df.gmv <= Q3 + 1.5*IQR)]
Q1 = camera_df.product_mrp.quantile(0.25)
Q3 = camera_df.product_mrp.quantile(0.75)
IQR = Q3 - Q1
camera_df = camera_df[(camera_df.product_mrp >= Q1 - 1.5*IQR) & (camera_df.product_mrp <= Q3 + 1.5*IQR)]

In [140]:
### Outlier Analysis
fig, axs = plt.subplots(1,3, figsize = (20,4))
plt1 = sns.boxplot(camera_df['gmv'], ax = axs[0])
plt2 = sns.boxplot(camera_df['units'], ax = axs[2])
plt4 = sns.boxplot(camera_df['product_mrp'], ax = axs[1])
plt.tight_layout()

In [141]:
camera_df.columns

Index(['fsn_id', 'order_date', 'Year', 'Month', 'order_id', 'order_item_id',
       'gmv', 'units', 'order_payment_type', 'sla',
       'product_analytic_category', 'product_analytic_sub_category',
       'product_mrp', 'product_procurement_sla', 'listing_price', 'discount',
       'Order_Item_Value', 'week', 'premium_product'],
      dtype='object')

In [142]:
camera_df.head()

,fsn_id,order_date,Year,Month,order_id,order_item_id,gmv,units,order_payment_type,sla,product_analytic_category,product_analytic_sub_category,product_mrp,product_procurement_sla,listing_price,discount,Order_Item_Value,week,premium_product
3,ACCCX3S5AHMF55FV,2015-10-14 12:05:15,2015,10,4416592101738400.000,4416592101738400.000,1690.000,1,1,4,CameraAccessory,CameraAccessory,2099.000,3,1690.000,0.190,2099.000,16,0
4,ACCCX3S5AHMF55FV,2015-10-17 21:25:03,2015,10,4419525153426400.000,4419525153426400.000,1618.000,1,1,6,CameraAccessory,CameraAccessory,2099.000,3,1618.000,0.230,2099.000,16,0
16,ACCD38YG3UDFDG6D,2015-10-06 00:45:01,2015,10,2409274855504000.000,2409274855504000.000,1495.000,1,0,5,CameraAccessory,CameraAccessory,2750.000,1,1495.000,0.460,2750.000,15,0
17,ACCD38YG3UDFDG6D,2015-10-12 23:38:46,2015,10,4415284052002400.000,4415284052002400.000,1440.000,1,0,5,CameraAccessory,CameraAccessory,2750.000,1,1440.000,0.480,2750.000,16,0
18,ACCD38YG3UDFDG6D,2015-10-14 10:09:46,2015,10,1416516698047300.000,1416516698047300.000,1440.000,1,0,4,CameraAccessory,CameraAccessory,2750.000,1,1440.000,0.480,2750.000,16,0


In [143]:
### Aggregating dataset on weekly level

ca_week = pd.DataFrame(camera_df.groupby('week').agg({'gmv':'sum','listing_price':'mean',
                                                             'product_mrp':'mean','discount':'mean',
                                                             'sla':'mean','product_procurement_sla':'mean',
                                                             'fsn_id':pd.Series.nunique,'order_item_id':pd.Series.nunique,
                                                             'order_id': pd.Series.nunique,
                                                             'order_payment_type':'sum',
                                                            'premium_product':'sum'}))

ca_week.reset_index( inplace = True)

In [144]:
ca_week.head()

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_item_id,order_id,order_payment_type,premium_product
0,1,11151.000,428.885,1121.038,0.491,2.423,2.731,20,26,26,26,0
1,2,1228580.000,501.230,1013.379,0.448,7.253,2.578,540,2390,2345,753,0
2,3,1344416.000,512.763,1096.768,0.458,6.567,2.577,538,2570,2539,823,0
3,4,1346986.000,511.973,1086.319,0.445,6.611,2.613,529,2571,2534,781,0
4,5,913445.000,522.839,1085.847,0.443,6.533,2.614,442,1703,1675,470,0


In [145]:
### Sum of GMV / No of unique Orders

ca_week['AOV'] = ca_week['gmv']/ca_week['order_id']

In [146]:
ca_week['online_order_perc'] = ca_week['order_payment_type']*100/ca_week['order_item_id']


In [147]:
ca_week.week.unique()


array([ 1,  2,  3,  4,  5,  6,  7,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53], dtype=int64)

In [148]:
calendar.week.unique()


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53], dtype=int64)

In [149]:
ca_week['week'] = ca_week['week'].astype(int)
calendar['week'] = calendar['week'].astype(int)

In [150]:
ca_week = ca_week.merge(marketing, how = 'left', on = 'week')

In [151]:
ca_week = ca_week.merge(calendar, how = 'left', on = 'week')

In [152]:
ca_week.head()

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_item_id,order_id,...,Payday,max_temp_C,min_temp_C,mean_temp_C,heat_deg_days,cool_deg_days,total_rain_mm,total_snow_cm,total_precip_mm,snow_on_grnd_cm
0,1,11151.000,428.885,1121.038,0.491,2.423,2.731,20,26,26,...,0.200,25.500,13.200,19.360,0.100,1.460,0.000,0.000,0.000,0.000
1,2,1228580.000,501.230,1013.379,0.448,7.253,2.578,540,2390,2345,...,0.000,24.025,14.385,19.263,1.059,2.322,4.155,0.029,4.184,0.041
2,3,1344416.000,512.763,1096.768,0.458,6.567,2.577,538,2570,2539,...,0.143,27.383,16.314,21.906,0.816,4.722,1.569,0.029,1.598,0.041
3,4,1346986.000,511.973,1086.319,0.445,6.611,2.613,529,2571,2534,...,0.000,28.240,14.342,20.540,1.632,4.172,1.141,0.029,1.170,0.041
4,5,913445.000,522.839,1085.847,0.443,6.533,2.614,442,1703,1675,...,0.143,29.811,16.242,23.092,0.816,5.908,4.341,0.029,4.370,0.041


## Exploratory Data Analysis for Camera Accessory

In [153]:
ca_week_viz = ca_week.round(2)

### Univariate Analysis

Target Variable

In [154]:
sns.distplot(ca_week_viz['gmv'],kde=True)

<AxesSubplot:xlabel='gmv', ylabel='Density'>

In [155]:
plt.figure(figsize=(15, 5))
sns.barplot(ca_week_viz['week'],ca_week_viz['gmv'])

<AxesSubplot:xlabel='week', ylabel='gmv'>

## Marketing KPI

In [156]:
ca_week_viz.columns

Index(['week', 'gmv', 'listing_price', 'product_mrp', 'discount', 'sla',
       'product_procurement_sla', 'fsn_id', 'order_item_id', 'order_id',
       'order_payment_type', 'premium_product', 'AOV', 'online_order_perc',
       'Date', 'Year', 'Month', 'Total_Investment', 'TV', 'Digital',
       'Sponsorship', 'Content_marketing', 'Online_marketing', 'Affiliates',
       'SEM', 'Radio', 'Other', 'TV_ads', 'Digital_ads', 'Sponsorship_ads',
       'Content_marketing_ads', 'Online_marketing_ads', 'Affiliates_ads',
       'SEM_ads', 'Radio_ads', 'Other_ads', 'NPS', 'Stock_Index',
       'Special_sales', 'Payday', 'max_temp_C', 'min_temp_C', 'mean_temp_C',
       'heat_deg_days', 'cool_deg_days', 'total_rain_mm', 'total_snow_cm',
       'total_precip_mm', 'snow_on_grnd_cm'],
      dtype='object')

In [157]:
fig, axs = plt.subplots(2,4,figsize=(16,8))

plt1 = sns.scatterplot(x = 'Total_Investment', y = 'gmv', data = ca_week_viz, ax = axs[0,0])

plt2 = sns.scatterplot(x = 'TV', y = 'gmv', data = ca_week_viz, ax = axs[0,1])

plt3 = sns.scatterplot(x = 'Digital', y = 'gmv', data = ca_week_viz, ax = axs[0,2])

plt4 = sns.scatterplot(x = 'Sponsorship', y = 'gmv', data = ca_week_viz, ax = axs[0,3])

plt5 = sns.scatterplot(x = 'Content_marketing', y = 'gmv', data = ca_week_viz, ax = axs[1,0])

plt6 = sns.scatterplot(x = 'Online_marketing', y = 'gmv', data = ca_week_viz, ax = axs[1,1])

plt7 = sns.scatterplot(x = 'Affiliates', y = 'gmv', data = ca_week_viz, ax = axs[1,2])

plt8 = sns.scatterplot(x = 'SEM', y = 'gmv', data = ca_week_viz, ax = axs[1,3])

plt.tight_layout()

### Bivariate Analysis

In [158]:
plt.figure(figsize=(20, 5))
sns.barplot(x= ca_week_viz['week'], y =ca_week_viz['gmv'], hue = ca_week_viz['Special_sales'], dodge = False)
plt.show()

### GMV and Dicount weekly

In [159]:
plt.figure(figsize=(20, 5))
sns.barplot(x= ca_week_viz['week'], y =ca_week_viz['gmv'], hue = pd.cut(ca_week_viz['discount'],3), dodge = False)
plt.show()

### Camera Accessory - Moving Average

In [160]:
### ca_week

### Moving Average for listing_price and discount

### ca_week = ca_week.sort_values('order_date')

ca_week[['MA2_LP','MA2_Discount']] = ca_week[['listing_price','discount']].rolling(window=2,min_periods=1).mean()
ca_week[['MA3_LP','MA3_Discount']] = ca_week[['listing_price','discount']].rolling(window=3,min_periods=1).mean()
ca_week[['MA4_LP','MA4_Discount']] = ca_week[['listing_price','discount']].rolling(window=4,min_periods=1).mean()

### Reference listed price Inflation 

ca_week['MA2_listed_price'] = (ca_week['listing_price']-ca_week['MA2_LP'])/ca_week['MA2_LP']
ca_week['MA3_listed_price'] = (ca_week['listing_price']-ca_week['MA3_LP'])/ca_week['MA3_LP']
ca_week['MA4_listed_price'] = (ca_week['listing_price']-ca_week['MA4_LP'])/ca_week['MA4_LP']

### Reference discount Inflation

ca_week['MA2_discount_offer'] = (ca_week['discount']-ca_week['MA2_Discount'])/ca_week['MA2_Discount']
ca_week['MA3_discount_offer'] = (ca_week['discount']-ca_week['MA3_Discount'])/ca_week['MA3_Discount']
ca_week['MA4_discount_offer'] = (ca_week['discount']-ca_week['MA4_Discount'])/ca_week['MA4_Discount']


ca_week.drop(['MA2_LP','MA3_LP','MA4_LP','MA2_Discount','MA3_Discount','MA4_Discount'], axis = 1, inplace = True)  
ca_week.head()

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_item_id,order_id,...,total_rain_mm,total_snow_cm,total_precip_mm,snow_on_grnd_cm,MA2_listed_price,MA3_listed_price,MA4_listed_price,MA2_discount_offer,MA3_discount_offer,MA4_discount_offer
0,1,11151.000,428.885,1121.038,0.491,2.423,2.731,20,26,26,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,2,1228580.000,501.230,1013.379,0.448,7.253,2.578,540,2390,2345,...,4.155,0.029,4.184,0.041,0.078,0.078,0.078,-0.046,-0.046,-0.046
2,3,1344416.000,512.763,1096.768,0.458,6.567,2.577,538,2570,2539,...,1.569,0.029,1.598,0.041,0.011,0.066,0.066,0.011,-0.017,-0.017
3,4,1346986.000,511.973,1086.319,0.445,6.611,2.613,529,2571,2534,...,1.141,0.029,1.170,0.041,-0.001,0.007,0.048,-0.014,-0.012,-0.033
4,5,913445.000,522.839,1085.847,0.443,6.533,2.614,442,1703,1675,...,4.341,0.029,4.370,0.041,0.011,0.014,0.021,-0.002,-0.012,-0.012


### Camera Accessory - Data Profiling to see multicollinearity and variable distributions

In [161]:
# ### To identify multicollinearity between variable
plt.figure(figsize=(20,20))
sns.heatmap(ca_week.corr(),annot = True, cmap="GnBu")
plt.show()

In [162]:
### Highly Correlated Columns should be dropped

ca_week.drop(['TV', 'Digital', 'Sponsorship', 'Content_marketing','Online_marketing', 'Affiliates', 'SEM','Radio',
              'Other'], axis = 1, inplace = True)

In [163]:
plt.figure(figsize=(25,20))
sns.heatmap(ca_week.corr(), cmap="coolwarm", annot=True)
plt.show()

In [164]:
ca_week.drop(['Affiliates_ads','SEM_ads','Digital_ads','Radio_ads','Other_ads','mean_temp_C','min_temp_C',
              'order_id','order_item_id','total_precip_mm','Total_Investment','MA3_discount_offer',
               'MA3_listed_price','AOV','max_temp_C','MA2_listed_price','MA4_discount_offer'],1,inplace=True)

In [165]:
#Successfully removed more than 90% correlation

### Camera Accessory - Lag Variable Functions

In [166]:
### Lag of listed_price, discount_offer, NPS, Special_sales

ca_week['lag_1_listed_price'] = ca_week['listing_price'].shift(-1).fillna(0)
ca_week['lag_2_listed_price'] = ca_week['listing_price'].shift(-2).fillna(0)
ca_week['lag_3_listed_price'] = ca_week['listing_price'].shift(-3).fillna(0)

ca_week['lag_1_discount'] = ca_week['discount'].shift(-1).fillna(0)
ca_week['lag_2_discount'] = ca_week['discount'].shift(-2).fillna(0)
ca_week['lag_3_discount'] = ca_week['discount'].shift(-3).fillna(0)

ca_week['lag_1_Stock_Index'] = ca_week['Stock_Index'].shift(-1).fillna(0)
ca_week['lag_2_Stock_Index'] = ca_week['Stock_Index'].shift(-2).fillna(0)
ca_week['lag_3_Stock_Index'] = ca_week['Stock_Index'].shift(-3).fillna(0)

ca_week['lag_1_Special_sales'] = ca_week['Special_sales'].shift(-1).fillna(0)
ca_week['lag_2_Special_sales'] = ca_week['Special_sales'].shift(-2).fillna(0)
ca_week['lag_3_Special_sales'] = ca_week['Special_sales'].shift(-3).fillna(0)

ca_week['lag_1_Payday'] = ca_week['Payday'].shift(-1).fillna(0)
ca_week['lag_2_Payday'] = ca_week['Payday'].shift(-2).fillna(0)
ca_week['lag_3_Payday'] = ca_week['Payday'].shift(-3).fillna(0)

ca_week['lag_1_NPS'] = ca_week['NPS'].shift(-1).fillna(0)
ca_week['lag_2_NPS'] = ca_week['NPS'].shift(-2).fillna(0)
ca_week['lag_3_NPS'] = ca_week['NPS'].shift(-3).fillna(0)

In [167]:
ca_week.head()

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_payment_type,premium_product,...,lag_3_Stock_Index,lag_1_Special_sales,lag_2_Special_sales,lag_3_Special_sales,lag_1_Payday,lag_2_Payday,lag_3_Payday,lag_1_NPS,lag_2_NPS,lag_3_NPS
0,1,11151.000,428.885,1121.038,0.491,2.423,2.731,20,26,0,...,1177.000,0.000,0.286,0.000,0.000,0.143,0.000,54.600,54.600,54.600
1,2,1228580.000,501.230,1013.379,0.448,7.253,2.578,540,753,0,...,1185.286,0.286,0.000,0.000,0.143,0.000,0.143,54.600,54.600,56.139
2,3,1344416.000,512.763,1096.768,0.458,6.567,2.577,538,823,0,...,1206.000,0.000,0.000,0.000,0.000,0.143,0.000,54.600,56.139,59.987
3,4,1346986.000,511.973,1086.319,0.445,6.611,2.613,529,781,0,...,1206.000,0.000,0.000,0.286,0.143,0.000,0.143,56.139,59.987,59.987
4,5,913445.000,522.839,1085.847,0.443,6.533,2.614,442,470,0,...,1206.000,0.000,0.286,0.429,0.000,0.143,0.000,59.987,59.987,59.987


# 2. Gaming Sub Category

In [168]:
gaming_accessory = consumer[consumer['product_analytic_sub_category'] == 'GamingAccessory']

In [169]:
###  Removing outliers is important as
###  1. There may be some garbage value.
###  2. Bulk orders can skew the analysis

In [170]:
### Outlier Analysis
fig, axs = plt.subplots(1,3, figsize = (20,4))
plt1 = sns.boxplot(gaming_accessory['gmv'], ax = axs[0])
plt2 = sns.boxplot(gaming_accessory['units'], ax = axs[2])
plt4 = sns.boxplot(gaming_accessory['product_mrp'], ax = axs[1])
plt.tight_layout()

In [171]:
### Treating outliers
### Outlier treatment for gmv & product_mrp
Q1 = gaming_accessory.gmv.quantile(0.25)
Q3 = gaming_accessory.gmv.quantile(0.75)
IQR = Q3 - Q1
gaming_accessory = gaming_accessory[(gaming_accessory.gmv >= Q1 - 1.5*IQR) & (gaming_accessory.gmv <= Q3 + 1.5*IQR)]
Q1 = gaming_accessory.product_mrp.quantile(0.25)
Q3 = gaming_accessory.product_mrp.quantile(0.75)
IQR = Q3 - Q1
gaming_accessory = gaming_accessory[(gaming_accessory.product_mrp >= Q1 - 1.5*IQR) & (gaming_accessory.product_mrp <= Q3 + 1.5*IQR)]

In [172]:
### Outlier Analysis
fig, axs = plt.subplots(1,3, figsize = (20,4))
plt1 = sns.boxplot(gaming_accessory['gmv'], ax = axs[0])
plt2 = sns.boxplot(gaming_accessory['units'], ax = axs[2])
plt4 = sns.boxplot(gaming_accessory['product_mrp'], ax = axs[1])
plt.tight_layout()

In [173]:
gaming_accessory.columns

Index(['fsn_id', 'order_date', 'Year', 'Month', 'order_id', 'order_item_id',
       'gmv', 'units', 'order_payment_type', 'sla',
       'product_analytic_category', 'product_analytic_sub_category',
       'product_mrp', 'product_procurement_sla', 'listing_price', 'discount',
       'Order_Item_Value', 'week', 'premium_product'],
      dtype='object')

In [174]:
### Aggregating dataset on weekly level

ga_week = pd.DataFrame(gaming_accessory.groupby('week').agg({'gmv':'sum','listing_price':'mean',
                                                             'product_mrp':'mean','discount':'mean',
                                                             'sla':'mean','product_procurement_sla':'mean',
                                                             'fsn_id':pd.Series.nunique,'order_item_id':pd.Series.nunique,
                                                             'order_id': pd.Series.nunique,
                                                             'order_payment_type':'sum'}))

ga_week.reset_index( inplace = True)

In [175]:
ga_week.head()

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_item_id,order_id,order_payment_type
0,1,17466.000,681.080,898.440,0.220,2.960,2.560,19,25,25,25
1,2,1435300.000,528.929,963.907,0.414,5.170,3.013,211,2629,2494,689
2,3,1097388.359,449.826,759.691,0.429,5.115,2.514,216,2382,2356,534
3,4,1158166.000,559.134,889.058,0.352,5.179,2.338,232,2023,1990,425
4,5,742062.000,554.214,886.076,0.363,5.552,2.325,224,1303,1287,246


In [176]:
### Sum of GMV / No of unique Orders

ga_week['AOV'] = ga_week['gmv']/ga_week['order_id']

In [177]:
ga_week['online_order_perc'] = ga_week['order_payment_type']*100/ga_week['order_item_id']

In [178]:
ga_week.head()

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_item_id,order_id,order_payment_type,AOV,online_order_perc
0,1,17466.000,681.080,898.440,0.220,2.960,2.560,19,25,25,25,698.640,100.000
1,2,1435300.000,528.929,963.907,0.414,5.170,3.013,211,2629,2494,689,575.501,26.208
2,3,1097388.359,449.826,759.691,0.429,5.115,2.514,216,2382,2356,534,465.785,22.418
3,4,1158166.000,559.134,889.058,0.352,5.179,2.338,232,2023,1990,425,581.993,21.008
4,5,742062.000,554.214,886.076,0.363,5.552,2.325,224,1303,1287,246,576.583,18.880


In [179]:
ga_week = ga_week.merge(marketing, how = 'left', on = 'week')

In [180]:
ga_week = ga_week.merge(calendar, how = 'left', on = 'week')

In [181]:
ga_week.head()

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_item_id,order_id,...,Payday,max_temp_C,min_temp_C,mean_temp_C,heat_deg_days,cool_deg_days,total_rain_mm,total_snow_cm,total_precip_mm,snow_on_grnd_cm
0,1,17466.000,681.080,898.440,0.220,2.960,2.560,19,25,25,...,0.200,25.500,13.200,19.360,0.100,1.460,0.000,0.000,0.000,0.000
1,2,1435300.000,528.929,963.907,0.414,5.170,3.013,211,2629,2494,...,0.000,24.025,14.385,19.263,1.059,2.322,4.155,0.029,4.184,0.041
2,3,1097388.359,449.826,759.691,0.429,5.115,2.514,216,2382,2356,...,0.143,27.383,16.314,21.906,0.816,4.722,1.569,0.029,1.598,0.041
3,4,1158166.000,559.134,889.058,0.352,5.179,2.338,232,2023,1990,...,0.000,28.240,14.342,20.540,1.632,4.172,1.141,0.029,1.170,0.041
4,5,742062.000,554.214,886.076,0.363,5.552,2.325,224,1303,1287,...,0.143,29.811,16.242,23.092,0.816,5.908,4.341,0.029,4.370,0.041


## Exploratory Data Analysis for Gaming Accessory

In [182]:
ga_week_viz = ga_week.round(2)

### Univariate Analysis

Target Variable

In [183]:
sns.distplot(ga_week_viz['gmv'],kde=True)

<AxesSubplot:xlabel='gmv', ylabel='Density'>

In [184]:
plt.figure(figsize=(15, 5))
sns.barplot(ga_week_viz['week'],ga_week_viz['gmv'])

<AxesSubplot:xlabel='week', ylabel='gmv'>

### Marketing KPI

In [185]:
ga_week_viz.columns

Index(['week', 'gmv', 'listing_price', 'product_mrp', 'discount', 'sla',
       'product_procurement_sla', 'fsn_id', 'order_item_id', 'order_id',
       'order_payment_type', 'AOV', 'online_order_perc', 'Date', 'Year',
       'Month', 'Total_Investment', 'TV', 'Digital', 'Sponsorship',
       'Content_marketing', 'Online_marketing', 'Affiliates', 'SEM', 'Radio',
       'Other', 'TV_ads', 'Digital_ads', 'Sponsorship_ads',
       'Content_marketing_ads', 'Online_marketing_ads', 'Affiliates_ads',
       'SEM_ads', 'Radio_ads', 'Other_ads', 'NPS', 'Stock_Index',
       'Special_sales', 'Payday', 'max_temp_C', 'min_temp_C', 'mean_temp_C',
       'heat_deg_days', 'cool_deg_days', 'total_rain_mm', 'total_snow_cm',
       'total_precip_mm', 'snow_on_grnd_cm'],
      dtype='object')

In [186]:
fig, axs = plt.subplots(2,4,figsize=(16,8))

plt1 = sns.scatterplot(x = 'Total_Investment', y = 'gmv', data = ga_week_viz, ax = axs[0,0])

plt2 = sns.scatterplot(x = 'TV', y = 'gmv', data = ga_week_viz, ax = axs[0,1])

plt3 = sns.scatterplot(x = 'Digital', y = 'gmv', data = ga_week_viz, ax = axs[0,2])

plt4 = sns.scatterplot(x = 'Sponsorship', y = 'gmv', data = ga_week_viz, ax = axs[0,3])

plt5 = sns.scatterplot(x = 'Content_marketing', y = 'gmv', data = ga_week_viz, ax = axs[1,0])

plt6 = sns.scatterplot(x = 'Online_marketing', y = 'gmv', data = ga_week_viz, ax = axs[1,1])

plt7 = sns.scatterplot(x = 'Affiliates', y = 'gmv', data = ga_week_viz, ax = axs[1,2])

plt8 = sns.scatterplot(x = 'SEM', y = 'gmv', data = ga_week_viz, ax = axs[1,3])

plt.tight_layout()

### Bivariate Analysis
GMV and Holiday weekly

In [187]:
plt.figure(figsize=(20, 5))
sns.barplot(x= ga_week_viz['week'], y =ga_week_viz['gmv'], hue = ga_week_viz['Special_sales'], dodge = False)
plt.show()

### GMV and Holiday weekly

In [188]:
plt.figure(figsize=(20, 5))
sns.barplot(x= ga_week_viz['week'], y =ga_week_viz['gmv'], hue = pd.cut(ga_week_viz['discount'],3), dodge = False)
plt.show()

### Gaming Accessory - Moving Average

In [189]:
### ga_week

### Moving Average for listed_price and discount_offer

### ga_week = ga_week.sort_values('order_date')

ga_week[['MA2_LP','MA2_Discount']] = ga_week[['listing_price','discount']].rolling(window=2,min_periods=1).mean()
ga_week[['MA3_LP','MA3_Discount']] = ga_week[['listing_price','discount']].rolling(window=3,min_periods=1).mean()
ga_week[['MA4_LP','MA4_Discount']] = ga_week[['listing_price','discount']].rolling(window=4,min_periods=1).mean()

### Reference listed price Inflation 

ga_week['MA2_listed_price'] = (ga_week['listing_price']-ga_week['MA2_LP'])/ga_week['MA2_LP']
ga_week['MA3_listed_price'] = (ga_week['listing_price']-ga_week['MA3_LP'])/ga_week['MA3_LP']
ga_week['MA4_listed_price'] = (ga_week['listing_price']-ga_week['MA4_LP'])/ga_week['MA4_LP']

### Reference discount Inflation

ga_week['MA2_discount'] = (ga_week['discount']-ga_week['MA2_Discount'])/ga_week['MA2_Discount']
ga_week['MA3_discount'] = (ga_week['discount']-ga_week['MA3_Discount'])/ga_week['MA3_Discount']
ga_week['MA4_discount'] = (ga_week['discount']-ga_week['MA4_Discount'])/ga_week['MA4_Discount']


ga_week.drop(['MA2_LP','MA3_LP','MA4_LP','MA2_Discount','MA3_Discount','MA4_Discount'], axis = 1, inplace = True)  
ga_week

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_item_id,order_id,...,total_rain_mm,total_snow_cm,total_precip_mm,snow_on_grnd_cm,MA2_listed_price,MA3_listed_price,MA4_listed_price,MA2_discount,MA3_discount,MA4_discount
0,1,17466.000,681.080,898.440,0.220,2.960,2.560,19,25,25,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,2,1435300.000,528.929,963.907,0.414,5.170,3.013,211,2629,2494,...,4.155,0.029,4.184,0.041,-0.126,-0.126,-0.126,0.306,0.306,0.306
2,3,1097388.359,449.826,759.691,0.429,5.115,2.514,216,2382,2356,...,1.569,0.029,1.598,0.041,-0.081,-0.187,-0.187,0.018,0.211,0.211
3,4,1158166.000,559.134,889.058,0.352,5.179,2.338,232,2023,1990,...,1.141,0.029,1.170,0.041,0.108,0.091,0.008,-0.098,-0.115,-0.003
4,5,742062.000,554.214,886.076,0.363,5.552,2.325,224,1303,1287,...,4.341,0.029,4.370,0.041,-0.004,0.064,0.060,0.015,-0.047,-0.067
5,6,391.000,195.500,362.000,0.535,4.500,2.000,2,2,2,...,0.938,0.058,0.997,0.082,-0.478,-0.552,-0.555,0.191,0.283,0.274
6,7,1611.000,805.500,1172.000,0.255,4.500,2.000,2,2,2,...,2.764,0.088,2.852,0.124,0.609,0.554,0.524,-0.354,-0.337,-0.323
7,8,359.000,359.000,699.000,0.490,3.000,3.000,1,1,1,...,4.769,0.029,4.798,0.041,-0.383,-0.208,-0.250,0.315,0.148,0.193
8,9,2303.000,575.750,1035.750,0.450,6.000,3.250,4,4,4,...,1.846,0.146,1.992,0.206,0.232,-0.007,0.190,-0.043,0.130,0.040
9,10,1084240.500,464.002,746.372,0.380,5.080,2.507,306,1978,2270,...,9.026,0.029,9.055,0.041,-0.107,-0.005,-0.158,-0.084,-0.136,-0.034


### Heatmap to see multicollinearity

In [190]:
plt.figure(figsize=(25,20))

### Heatmap
sns.heatmap(ga_week.corr(), cmap="coolwarm", annot=True)
plt.show()

In [191]:
ga_week.drop(['TV', 'Digital', 'Sponsorship', 'Content_marketing','Online_marketing', 'Affiliates', 'SEM','Radio',
              'Other','Affiliates_ads','SEM_ads','Digital_ads','Radio_ads','Other_ads','mean_temp_C','min_temp_C',
              'order_id','order_item_id','total_precip_mm','Total_Investment','MA3_discount',
              'MA3_listed_price','AOV','MA4_listed_price'], axis = 1, inplace = True)

In [192]:
ga_week.drop(['max_temp_C'], axis = 1, inplace = True)

In [193]:
###  Successfully removed more than 90% highly correlated variables from dataset.

### Gaming Accessory - Lag Variable Functions

In [194]:
### Lag of listed_price, discount_offer, NPS, Special_sales

ga_week['lag_1_listed_price'] = ga_week['listing_price'].shift(-1).fillna(0)
ga_week['lag_2_listed_price'] = ga_week['listing_price'].shift(-2).fillna(0)
ga_week['lag_3_listed_price'] = ga_week['listing_price'].shift(-3).fillna(0)

ga_week['lag_1_discount_offer'] = ga_week['discount'].shift(-1).fillna(0)
ga_week['lag_2_discount_offer'] = ga_week['discount'].shift(-2).fillna(0)
ga_week['lag_3_discount_offer'] = ga_week['discount'].shift(-3).fillna(0)

ga_week['lag_1_NPS'] = ga_week['NPS'].shift(-1).fillna(0)
ga_week['lag_2_NPS'] = ga_week['NPS'].shift(-2).fillna(0)
ga_week['lag_3_NPS'] = ga_week['NPS'].shift(-3).fillna(0)

ga_week['lag_1_Stock_Index'] = ga_week['Stock_Index'].shift(-1).fillna(0)
ga_week['lag_2_Stock_Index'] = ga_week['Stock_Index'].shift(-2).fillna(0)
ga_week['lag_3_Stock_Index'] = ga_week['Stock_Index'].shift(-3).fillna(0)

ga_week['lag_1_Special_sales'] = ga_week['Special_sales'].shift(-1).fillna(0)
ga_week['lag_2_Special_sales'] = ga_week['Special_sales'].shift(-2).fillna(0)
ga_week['lag_3_Special_sales'] = ga_week['Special_sales'].shift(-3).fillna(0)

ga_week['lag_1_Payday'] = ga_week['Payday'].shift(-1).fillna(0)
ga_week['lag_2_Payday'] = ga_week['Payday'].shift(-2).fillna(0)
ga_week['lag_3_Payday'] = ga_week['Payday'].shift(-3).fillna(0)

ga_week

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_payment_type,online_order_perc,...,lag_3_NPS,lag_1_Stock_Index,lag_2_Stock_Index,lag_3_Stock_Index,lag_1_Special_sales,lag_2_Special_sales,lag_3_Special_sales,lag_1_Payday,lag_2_Payday,lag_3_Payday
0,1,17466.000,681.080,898.440,0.220,2.960,2.560,19,25,100.000,...,54.600,1177.000,1177.000,1177.000,0.000,0.286,0.000,0.000,0.143,0.000
1,2,1435300.000,528.929,963.907,0.414,5.170,3.013,211,689,26.208,...,56.139,1177.000,1177.000,1185.286,0.286,0.000,0.000,0.143,0.000,0.143
2,3,1097388.359,449.826,759.691,0.429,5.115,2.514,216,534,22.418,...,59.987,1177.000,1185.286,1206.000,0.000,0.000,0.000,0.000,0.143,0.000
3,4,1158166.000,559.134,889.058,0.352,5.179,2.338,232,425,21.008,...,59.987,1185.286,1206.000,1206.000,0.000,0.000,0.286,0.143,0.000,0.143
4,5,742062.000,554.214,886.076,0.363,5.552,2.325,224,246,18.880,...,59.987,1206.000,1206.000,1206.000,0.000,0.286,0.143,0.000,0.143,0.000
5,6,391.000,195.500,362.000,0.535,4.500,2.000,2,0,0.000,...,59.987,1206.000,1206.000,1206.000,0.286,0.143,0.429,0.143,0.000,0.000
6,7,1611.000,805.500,1172.000,0.255,4.500,2.000,2,0,0.000,...,48.791,1206.000,1206.000,1116.000,0.143,0.429,0.000,0.000,0.000,0.143
7,8,359.000,359.000,699.000,0.490,3.000,3.000,1,0,0.000,...,46.925,1206.000,1116.000,1101.000,0.429,0.000,0.000,0.000,0.143,0.000
8,9,2303.000,575.750,1035.750,0.450,6.000,3.250,4,0,0.000,...,46.925,1116.000,1101.000,1101.000,0.000,0.000,0.000,0.143,0.000,0.143
9,10,1084240.500,464.002,746.372,0.380,5.080,2.507,306,442,22.346,...,46.925,1101.000,1101.000,1101.000,0.000,0.000,0.000,0.000,0.143,0.000


In [195]:
ga_week.head()

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_payment_type,online_order_perc,...,lag_3_NPS,lag_1_Stock_Index,lag_2_Stock_Index,lag_3_Stock_Index,lag_1_Special_sales,lag_2_Special_sales,lag_3_Special_sales,lag_1_Payday,lag_2_Payday,lag_3_Payday
0,1,17466.000,681.080,898.440,0.220,2.960,2.560,19,25,100.000,...,54.600,1177.000,1177.000,1177.000,0.000,0.286,0.000,0.000,0.143,0.000
1,2,1435300.000,528.929,963.907,0.414,5.170,3.013,211,689,26.208,...,56.139,1177.000,1177.000,1185.286,0.286,0.000,0.000,0.143,0.000,0.143
2,3,1097388.359,449.826,759.691,0.429,5.115,2.514,216,534,22.418,...,59.987,1177.000,1185.286,1206.000,0.000,0.000,0.000,0.000,0.143,0.000
3,4,1158166.000,559.134,889.058,0.352,5.179,2.338,232,425,21.008,...,59.987,1185.286,1206.000,1206.000,0.000,0.000,0.286,0.143,0.000,0.143
4,5,742062.000,554.214,886.076,0.363,5.552,2.325,224,246,18.880,...,59.987,1206.000,1206.000,1206.000,0.000,0.286,0.143,0.000,0.143,0.000


# 3. Home Audio Sub Category

In [196]:
home_audio = consumer[consumer['product_analytic_sub_category'] == 'HomeAudio']

In [197]:
###  Removing outliers is important as
###  1. There may be some garbage value.
###  2. Bulk orders can skew the analysis

In [198]:
### Outlier Analysis
fig, axs = plt.subplots(1,3, figsize = (20,4))
plt1 = sns.boxplot(home_audio['gmv'], ax = axs[0])
plt2 = sns.boxplot(home_audio['units'], ax = axs[2])
plt4 = sns.boxplot(home_audio['product_mrp'], ax = axs[1])
plt.tight_layout()

In [199]:
### Treating outliers
### Outlier treatment for gmv & product_mrp
Q1 = home_audio.gmv.quantile(0.25)
Q3 = home_audio.gmv.quantile(0.75)
IQR = Q3 - Q1
home_audio = home_audio[(home_audio.gmv >= Q1 - 1.5*IQR) & (home_audio.gmv <= Q3 + 1.5*IQR)]
Q1 = home_audio.product_mrp.quantile(0.25)
Q3 = home_audio.product_mrp.quantile(0.75)
IQR = Q3 - Q1
home_audio = home_audio[(home_audio.product_mrp >= Q1 - 1.5*IQR) & (home_audio.product_mrp <= Q3 + 1.5*IQR)]

In [200]:
### Outlier Analysis
fig, axs = plt.subplots(1,3, figsize = (20,4))
plt1 = sns.boxplot(home_audio['gmv'], ax = axs[0])
plt2 = sns.boxplot(home_audio['units'], ax = axs[2])
plt4 = sns.boxplot(home_audio['product_mrp'], ax = axs[1])
plt.tight_layout()

In [201]:
home_audio.columns

Index(['fsn_id', 'order_date', 'Year', 'Month', 'order_id', 'order_item_id',
       'gmv', 'units', 'order_payment_type', 'sla',
       'product_analytic_category', 'product_analytic_sub_category',
       'product_mrp', 'product_procurement_sla', 'listing_price', 'discount',
       'Order_Item_Value', 'week', 'premium_product'],
      dtype='object')

In [202]:
### Aggregating dataset on weekly level

ha_week = pd.DataFrame(home_audio.groupby('week').agg({'gmv':'sum','listing_price':'mean',
                                                             'product_mrp':'mean','discount':'mean',
                                                             'sla':'mean','product_procurement_sla':'mean',
                                                             'fsn_id':pd.Series.nunique,'order_item_id':pd.Series.nunique,
                                                             'order_id': pd.Series.nunique,
                                                             'order_payment_type':'sum'}))

ha_week.reset_index( inplace = True)

In [203]:
ha_week.head()

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_item_id,order_id,order_payment_type
0,1,83742.000,2392.629,4369.057,0.397,2.943,3.486,19,35,35,34
1,2,3725167.133,1845.875,2971.504,0.326,7.454,2.751,202,2012,2011,464
2,3,4344419.000,1855.961,3121.880,0.339,7.116,2.666,227,2335,2330,527
3,4,3917329.000,1798.505,3008.836,0.328,7.168,2.762,227,2167,2160,450
4,5,2812144.000,1853.243,3094.751,0.328,7.297,2.663,200,1513,1511,327


In [204]:
### Sum of GMV / No of unique Orders

ha_week['AOV'] = ha_week['gmv']/ha_week['order_id']

In [205]:
ha_week['online_order_perc'] = ha_week['order_payment_type']*100/ha_week['order_item_id']

In [206]:
ha_week.head()

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_item_id,order_id,order_payment_type,AOV,online_order_perc
0,1,83742.000,2392.629,4369.057,0.397,2.943,3.486,19,35,35,34,2392.629,97.143
1,2,3725167.133,1845.875,2971.504,0.326,7.454,2.751,202,2012,2011,464,1852.395,23.062
2,3,4344419.000,1855.961,3121.880,0.339,7.116,2.666,227,2335,2330,527,1864.558,22.570
3,4,3917329.000,1798.505,3008.836,0.328,7.168,2.762,227,2167,2160,450,1813.578,20.766
4,5,2812144.000,1853.243,3094.751,0.328,7.297,2.663,200,1513,1511,327,1861.114,21.613


In [207]:
ha_week = ha_week.merge(marketing, how = 'left', on = 'week')

In [208]:
ha_week = ha_week.merge(calendar, how = 'left', on = 'week')

In [209]:
ha_week.head()

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_item_id,order_id,...,Payday,max_temp_C,min_temp_C,mean_temp_C,heat_deg_days,cool_deg_days,total_rain_mm,total_snow_cm,total_precip_mm,snow_on_grnd_cm
0,1,83742.000,2392.629,4369.057,0.397,2.943,3.486,19,35,35,...,0.200,25.500,13.200,19.360,0.100,1.460,0.000,0.000,0.000,0.000
1,2,3725167.133,1845.875,2971.504,0.326,7.454,2.751,202,2012,2011,...,0.000,24.025,14.385,19.263,1.059,2.322,4.155,0.029,4.184,0.041
2,3,4344419.000,1855.961,3121.880,0.339,7.116,2.666,227,2335,2330,...,0.143,27.383,16.314,21.906,0.816,4.722,1.569,0.029,1.598,0.041
3,4,3917329.000,1798.505,3008.836,0.328,7.168,2.762,227,2167,2160,...,0.000,28.240,14.342,20.540,1.632,4.172,1.141,0.029,1.170,0.041
4,5,2812144.000,1853.243,3094.751,0.328,7.297,2.663,200,1513,1511,...,0.143,29.811,16.242,23.092,0.816,5.908,4.341,0.029,4.370,0.041


## Exploratory Data Analysis of Home Audio

In [210]:
ha_week_viz = ha_week.round(2)

### Univariate Analysis
Target Variable

In [211]:
sns.distplot(ha_week_viz['gmv'],kde=True)

<AxesSubplot:xlabel='gmv', ylabel='Density'>

In [212]:
plt.figure(figsize=(15, 5))
sns.barplot(ha_week_viz['week'],ha_week_viz['gmv'])

<AxesSubplot:xlabel='week', ylabel='gmv'>

## Marketing KPI

In [213]:
ha_week_viz.columns

Index(['week', 'gmv', 'listing_price', 'product_mrp', 'discount', 'sla',
       'product_procurement_sla', 'fsn_id', 'order_item_id', 'order_id',
       'order_payment_type', 'AOV', 'online_order_perc', 'Date', 'Year',
       'Month', 'Total_Investment', 'TV', 'Digital', 'Sponsorship',
       'Content_marketing', 'Online_marketing', 'Affiliates', 'SEM', 'Radio',
       'Other', 'TV_ads', 'Digital_ads', 'Sponsorship_ads',
       'Content_marketing_ads', 'Online_marketing_ads', 'Affiliates_ads',
       'SEM_ads', 'Radio_ads', 'Other_ads', 'NPS', 'Stock_Index',
       'Special_sales', 'Payday', 'max_temp_C', 'min_temp_C', 'mean_temp_C',
       'heat_deg_days', 'cool_deg_days', 'total_rain_mm', 'total_snow_cm',
       'total_precip_mm', 'snow_on_grnd_cm'],
      dtype='object')

In [214]:
fig, axs = plt.subplots(2,4,figsize=(16,8))

plt1 = sns.scatterplot(x = 'Total_Investment', y = 'gmv', data = ha_week_viz, ax = axs[0,0])

plt2 = sns.scatterplot(x = 'TV', y = 'gmv', data = ha_week_viz, ax = axs[0,1])

plt3 = sns.scatterplot(x = 'Digital', y = 'gmv', data = ha_week_viz, ax = axs[0,2])

plt4 = sns.scatterplot(x = 'Sponsorship', y = 'gmv', data = ha_week_viz, ax = axs[0,3])

plt5 = sns.scatterplot(x = 'Content_marketing', y = 'gmv', data = ha_week_viz, ax = axs[1,0])

plt6 = sns.scatterplot(x = 'Online_marketing', y = 'gmv', data = ha_week_viz, ax = axs[1,1])

plt7 = sns.scatterplot(x = 'Affiliates', y = 'gmv', data = ha_week_viz, ax = axs[1,2])

plt8 = sns.scatterplot(x = 'SEM', y = 'gmv', data = ha_week_viz, ax = axs[1,3])

plt.tight_layout()

### Bivariate Analysis
GMV and Holiday weekly

In [215]:
plt.figure(figsize=(20, 5))
sns.barplot(x= ha_week_viz['week'], y =ha_week_viz['gmv'], hue = ha_week_viz['Special_sales'], dodge = False)
plt.show()

### GMV and discount weekly

In [216]:
plt.figure(figsize=(20, 5))
sns.barplot(x= ha_week_viz['week'], y =ha_week_viz['gmv'], hue = pd.cut(ha_week_viz['discount'],3), dodge = False)
plt.show()

In [217]:
### ha_week

### Moving Average for listed_price and discount_offer

### ha_week = ha_week.sort_values('order_date')

ha_week[['MA2_LP','MA2_Discount']] = ha_week[['listing_price','discount']].rolling(window=2,min_periods=1).mean()
ha_week[['MA3_LP','MA3_Discount']] = ha_week[['listing_price','discount']].rolling(window=3,min_periods=1).mean()
ha_week[['MA4_LP','MA4_Discount']] = ha_week[['listing_price','discount']].rolling(window=4,min_periods=1).mean()

### Reference listed price Inflation 

ha_week['MA2_listed_price'] = (ha_week['listing_price']-ha_week['MA2_LP'])/ha_week['MA2_LP']
ha_week['MA3_listed_price'] = (ha_week['listing_price']-ha_week['MA3_LP'])/ha_week['MA3_LP']
ha_week['MA4_listed_price'] = (ha_week['listing_price']-ha_week['MA4_LP'])/ha_week['MA4_LP']

### Reference discount Inflation

ha_week['MA2_discount'] = (ha_week['discount']-ha_week['MA2_Discount'])/ha_week['MA2_Discount']
ha_week['MA3_discount'] = (ha_week['discount']-ha_week['MA3_Discount'])/ha_week['MA3_Discount']
ha_week['MA4_discount'] = (ha_week['discount']-ha_week['MA4_Discount'])/ha_week['MA4_Discount']


ha_week.drop(['MA2_LP','MA3_LP','MA4_LP','MA2_Discount','MA3_Discount','MA4_Discount'], axis = 1, inplace = True)  
ha_week

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_item_id,order_id,...,total_rain_mm,total_snow_cm,total_precip_mm,snow_on_grnd_cm,MA2_listed_price,MA3_listed_price,MA4_listed_price,MA2_discount,MA3_discount,MA4_discount
0,1,83742.000,2392.629,4369.057,0.397,2.943,3.486,19,35,35,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,2,3725167.133,1845.875,2971.504,0.326,7.454,2.751,202,2012,2011,...,4.155,0.029,4.184,0.041,-0.129,-0.129,-0.129,-0.098,-0.098,-0.098
2,3,4344419.000,1855.961,3121.880,0.339,7.116,2.666,227,2335,2330,...,1.569,0.029,1.598,0.041,0.003,-0.086,-0.086,0.020,-0.042,-0.042
3,4,3917329.000,1798.505,3008.836,0.328,7.168,2.762,227,2167,2160,...,1.141,0.029,1.170,0.041,-0.016,-0.019,-0.089,-0.017,-0.010,-0.057
4,5,2812144.000,1853.243,3094.751,0.328,7.297,2.663,200,1513,1511,...,4.341,0.029,4.370,0.041,0.015,0.009,0.008,0.000,-0.011,-0.007
5,10,3091251.000,1768.029,3145.995,0.365,5.682,2.729,227,1581,1743,...,9.026,0.029,9.055,0.041,-0.024,-0.021,-0.028,0.053,0.072,0.072
6,11,3489765.000,1742.461,3146.834,0.367,5.603,2.821,241,1628,1997,...,1.424,0.058,1.482,0.082,-0.007,-0.025,-0.027,0.003,0.039,0.058
7,12,3091999.000,1708.961,2986.984,0.359,6.311,2.618,241,1798,1802,...,1.422,0.088,1.509,0.124,-0.010,-0.018,-0.033,-0.011,-0.013,0.012
8,13,3405749.000,1591.263,2734.433,0.349,6.399,2.589,248,2134,2134,...,0.738,0.058,0.797,0.082,-0.036,-0.053,-0.065,-0.014,-0.026,-0.031
9,14,3434064.000,1688.939,2875.302,0.343,6.448,2.444,247,2026,2026,...,3.395,0.058,3.454,0.082,0.030,0.016,0.004,-0.009,-0.021,-0.032


### Heatmap to see multicollinearity

In [218]:
plt.figure(figsize=(25,20))

### Heatmap
sns.heatmap(ha_week.corr(), cmap="coolwarm", annot=True)
plt.show()

In [219]:
ha_week.drop(['TV', 'Digital', 'Sponsorship', 'Content_marketing','Online_marketing', 'Affiliates', 'SEM','Radio',
              'Other','Affiliates_ads','SEM_ads','Digital_ads','Radio_ads','Other_ads','mean_temp_C','min_temp_C',
              'order_id','order_item_id','total_precip_mm','Total_Investment','MA3_discount',
              'MA3_listed_price','AOV'], axis = 1, inplace = True)

In [220]:
ha_week.drop(['max_temp_C'], axis = 1, inplace = True)

In [221]:
###  Successfully removed more than 90% highly correlated variables from dataset.

### Home Audio - Lag Variable Functions

In [222]:
### Lag of listed_price, discount_offer, NPS, Special_sales

ha_week['lag_1_listed_price'] = ha_week['listing_price'].shift(-1).fillna(0)
ha_week['lag_2_listed_price'] = ha_week['listing_price'].shift(-2).fillna(0)
ha_week['lag_3_listed_price'] = ha_week['listing_price'].shift(-3).fillna(0)

ha_week['lag_1_discount_offer'] = ha_week['discount'].shift(-1).fillna(0)
ha_week['lag_2_discount_offer'] = ha_week['discount'].shift(-2).fillna(0)
ha_week['lag_3_discount_offer'] = ha_week['discount'].shift(-3).fillna(0)

ha_week['lag_1_NPS'] = ha_week['NPS'].shift(-1).fillna(0)
ha_week['lag_2_NPS'] = ha_week['NPS'].shift(-2).fillna(0)
ha_week['lag_3_NPS'] = ha_week['NPS'].shift(-3).fillna(0)

ha_week['lag_1_Stock_Index'] = ha_week['Stock_Index'].shift(-1).fillna(0)
ha_week['lag_2_Stock_Index'] = ha_week['Stock_Index'].shift(-2).fillna(0)
ha_week['lag_3_Stock_Index'] = ha_week['Stock_Index'].shift(-3).fillna(0)

ha_week['lag_1_Special_sales'] = ha_week['Special_sales'].shift(-1).fillna(0)
ha_week['lag_2_Special_sales'] = ha_week['Special_sales'].shift(-2).fillna(0)
ha_week['lag_3_Special_sales'] = ha_week['Special_sales'].shift(-3).fillna(0)

ha_week['lag_1_Payday'] = ha_week['Payday'].shift(-1).fillna(0)
ha_week['lag_2_Payday'] = ha_week['Payday'].shift(-2).fillna(0)
ha_week['lag_3_Payday'] = ha_week['Payday'].shift(-3).fillna(0)
ha_week

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_payment_type,online_order_perc,...,lag_3_NPS,lag_1_Stock_Index,lag_2_Stock_Index,lag_3_Stock_Index,lag_1_Special_sales,lag_2_Special_sales,lag_3_Special_sales,lag_1_Payday,lag_2_Payday,lag_3_Payday
0,1,83742.000,2392.629,4369.057,0.397,2.943,3.486,19,34,97.143,...,54.600,1177.000,1177.000,1177.000,0.000,0.286,0.000,0.000,0.143,0.000
1,2,3725167.133,1845.875,2971.504,0.326,7.454,2.751,202,464,23.062,...,56.139,1177.000,1177.000,1185.286,0.286,0.000,0.000,0.143,0.000,0.143
2,3,4344419.000,1855.961,3121.880,0.339,7.116,2.666,227,527,22.570,...,48.791,1177.000,1185.286,1116.000,0.000,0.000,0.000,0.000,0.143,0.143
3,4,3917329.000,1798.505,3008.836,0.328,7.168,2.762,227,450,20.766,...,46.925,1185.286,1116.000,1101.000,0.000,0.000,0.000,0.143,0.143,0.000
4,5,2812144.000,1853.243,3094.751,0.328,7.297,2.663,200,327,21.613,...,46.925,1116.000,1101.000,1101.000,0.000,0.000,0.000,0.143,0.000,0.143
5,10,3091251.000,1768.029,3145.995,0.365,5.682,2.729,227,394,24.921,...,46.925,1101.000,1101.000,1101.000,0.000,0.000,0.000,0.000,0.143,0.000
6,11,3489765.000,1742.461,3146.834,0.367,5.603,2.821,241,393,24.140,...,45.481,1101.000,1101.000,1163.286,0.000,0.000,0.000,0.143,0.000,0.143
7,12,3091999.000,1708.961,2986.984,0.359,6.311,2.618,241,387,21.524,...,44.398,1101.000,1163.286,1210.000,0.000,0.000,0.000,0.000,0.143,0.000
8,13,3405749.000,1591.263,2734.433,0.349,6.399,2.589,248,448,20.993,...,44.398,1163.286,1210.000,1210.000,0.000,0.000,0.429,0.143,0.000,0.143
9,14,3434064.000,1688.939,2875.302,0.343,6.448,2.444,247,433,21.372,...,44.398,1210.000,1210.000,1210.000,0.000,0.429,0.000,0.000,0.143,0.000


In [223]:
ha_week.head(10)

,week,gmv,listing_price,product_mrp,discount,sla,product_procurement_sla,fsn_id,order_payment_type,online_order_perc,...,lag_3_NPS,lag_1_Stock_Index,lag_2_Stock_Index,lag_3_Stock_Index,lag_1_Special_sales,lag_2_Special_sales,lag_3_Special_sales,lag_1_Payday,lag_2_Payday,lag_3_Payday
0,1,83742.000,2392.629,4369.057,0.397,2.943,3.486,19,34,97.143,...,54.600,1177.000,1177.000,1177.000,0.000,0.286,0.000,0.000,0.143,0.000
1,2,3725167.133,1845.875,2971.504,0.326,7.454,2.751,202,464,23.062,...,56.139,1177.000,1177.000,1185.286,0.286,0.000,0.000,0.143,0.000,0.143
2,3,4344419.000,1855.961,3121.880,0.339,7.116,2.666,227,527,22.570,...,48.791,1177.000,1185.286,1116.000,0.000,0.000,0.000,0.000,0.143,0.143
3,4,3917329.000,1798.505,3008.836,0.328,7.168,2.762,227,450,20.766,...,46.925,1185.286,1116.000,1101.000,0.000,0.000,0.000,0.143,0.143,0.000
4,5,2812144.000,1853.243,3094.751,0.328,7.297,2.663,200,327,21.613,...,46.925,1116.000,1101.000,1101.000,0.000,0.000,0.000,0.143,0.000,0.143
5,10,3091251.000,1768.029,3145.995,0.365,5.682,2.729,227,394,24.921,...,46.925,1101.000,1101.000,1101.000,0.000,0.000,0.000,0.000,0.143,0.000
6,11,3489765.000,1742.461,3146.834,0.367,5.603,2.821,241,393,24.140,...,45.481,1101.000,1101.000,1163.286,0.000,0.000,0.000,0.143,0.000,0.143
7,12,3091999.000,1708.961,2986.984,0.359,6.311,2.618,241,387,21.524,...,44.398,1101.000,1163.286,1210.000,0.000,0.000,0.000,0.000,0.143,0.000
8,13,3405749.000,1591.263,2734.433,0.349,6.399,2.589,248,448,20.993,...,44.398,1163.286,1210.000,1210.000,0.000,0.000,0.429,0.143,0.000,0.143
9,14,3434064.000,1688.939,2875.302,0.343,6.448,2.444,247,433,21.372,...,44.398,1210.000,1210.000,1210.000,0.000,0.429,0.000,0.000,0.143,0.000


In [224]:
# ca_week.to_csv('ca_week.csv')

In [225]:
# ga_week.to_csv('ga_week.csv')

In [226]:
# ha_week.to_csv('ha_week.csv')